<a href="https://www.kaggle.com/code/oswind/stockchat-agents-edition?scriptVersionId=283096961" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Environment Setup

In [1]:
# Setup the notebook based on running environment.
import os
# Optional: Enable telemetry in browser_use and chromadb.
os.environ["ANONYMIZED_TELEMETRY"] = "false"
# Check for kaggle environment.
if os.getenv("KAGGLE_KERNEL_RUN_TYPE"):
    # Kaggle Run: update the system.
    !pip uninstall -qqy google-ai-generativelanguage pydrive2 tensorflow tensorflow-decision-forests cryptography pyOpenSSL langchain langchain-core nltk ray click google-generativeai google-cloud-translate datasets cesium bigframes plotnine mlxtend fastai spacy thinc google-colab gcsfs jupyter-kernel-gateway nltk preprocessing
    !pip install -qU posthog\<6.0.0 google-genai==1.50.0 chromadb==0.6.3 opentelemetry-proto==1.37.0
    !pip install -qU langchain-community langchain-text-splitters wikipedia lmnr[all] google-adk google-adk[eval] google-cloud-translate
    from kaggle_secrets import UserSecretsClient # type: ignore
    from jupyter_server.serverapp import list_running_servers # type: ignore
else:
    # Mock the kaggle secrets client.
    class UserSecretsClient:
        @classmethod
        def set_secret(cls, id: str, value: str):
            os.environ[id] = value
        @classmethod
        def get_secret(cls, id: str):
            try:
                return os.environ[id]
            except KeyError as e:
                print(f"KeyError: authentication token for {id} is undefined")
    # Local Run: update the venv.
    %pip install -qU posthog\<6.0.0 google-genai==1.50.0 chromadb==0.6.3 opentelemetry-proto==1.37.0
    %pip install -qU langchain-community langchain-text-splitters wikipedia pandas google-api-core "lmnr[all]" browser-use ollama google-adk "google-adk[eval]"
    from browser_use import Agent as BrowserAgent

import ast, chromadb, json, logging, pandas, platform, pytz, re, requests, sys, threading, time, warnings, wikipedia
from bs4 import Tag
from chromadb import Documents, Embeddings
from datetime import datetime, timedelta
from dateutil.parser import parse
from enum import Enum
from google.adk.apps.app import App
from google.adk.sessions import InMemorySessionService, BaseSessionService as SessionService, Session
from google.adk.runners import Runner, Event
from google import genai
from google.api_core import retry, exceptions
from google.genai.models import Models
from google.genai import types, errors
from IPython.display import Markdown, display, HTML
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_text_splitters.html import HTMLSemanticPreservingSplitter
from langchain_text_splitters.json import RecursiveJsonSplitter
from lmnr import Laminar
from math import inf
from pydantic import BaseModel, field_validator
from threading import Timer
from tqdm import tqdm
from typing import Optional, Callable, NewType, NamedTuple
from wikipedia.exceptions import DisambiguationError, PageError

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.3/257.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 60.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 56.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 66.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108

In [2]:
# Prepare the Gemini api for use.
# Setup a retry helper for generation not run through the below api-helper.
is_retriable = lambda e: (isinstance(e, errors.APIError) and e.code in {429, 503, 500})
Models.generate_content = retry.Retry(predicate=is_retriable)(Models.generate_content)
Models.embed_content = retry.Retry(predicate=is_retriable)(Models.embed_content)

# Activate Laminar auto-instrumentation.
try:
    Laminar.initialize(project_api_key=UserSecretsClient().get_secret("LMNR_PROJECT_API_KEY"))
except:
    print("Skipping Laminar.initialize()")

class GeminiModel:
    def __init__(self, rpm: list, tpm: list, rpd: list):
        self.rpm = rpm # requests per minute
        self.tpm = tpm # tokens per minute in millions
        self.rpd = rpd # requests per day
        self.err = [0,0] # validation, api_related

# A python api-helper with model fail-over/chaining/retry support.
GeminiEmbedFunction = NewType("GeminiEmbedFunction", None) # forward-decl
class Api:
    gen_limit_in = 1048576
    emb_limit_in = 2048
    gen_model = {
        "gemini-2.5-flash": GeminiModel([10,1000,2000,10000],[.25,1,3,8],[250,10000,100000,inf]), # stable: 10 RPM/250K TPM/250 RPD
        "gemini-2.5-flash-preview-09-2025": GeminiModel([10,1000,2000,10000],[.25,1,3,8],[250,10000,100000,inf]), # exp: 10 RPM/250K TPM/250 RPD
        "gemini-2.0-flash-exp": GeminiModel([10,10,10,10],[.25,.25,.25,.25],[200,500,500,500]), # latest w/thinking: 10 RPM/250K TPM/200 RPD
        "gemini-2.0-flash": GeminiModel([15,2000,10000,30000],[1,4,10,30],[200,inf,inf,inf]), # stable wo/thinking: 15 RPM/1M TPM/200 RPD
        "gemini-2.5-flash-lite": GeminiModel([15,4000,10000,30000],[.25,4,10,30],[1000,inf,inf,inf]), # stable: 15 RPM/250K TPM/1K RPD
        "gemini-2.5-flash-lite-preview-09-2025": GeminiModel([15,4000,10000,30000],[.25,4,10,30],[1000,inf,inf,inf]), # exp: 15 RPM/250K TPM/1K RPD
        "gemini-2.5-pro": GeminiModel([5,150,1000,2000],[.125,2,5,8],[100,10000,50000,inf]), # stable: 5 RPM/250K TPM/100 RPD
    }
    gen_local = ["gemma3n:e4b","gemma3:12b-it-qat"]
    default_local = 0
    default_model = []
    embed_model = "gemini-embedding-001", GeminiModel([100,3000,5000,10000],[.03,1,5,10],[1000,inf,inf,inf]) # stable: 100 RPM/30K TPM/1000 RPD/100 per batch
    embed_local = False
    error_total = 0
    min_rpm = 3
    min_tpm = 40000
    dt_between = 2.0
    errored = False
    running = False
    dt_err = 45.0
    dt_rpm = 60.0

    @classmethod
    def get(cls, url: str):
        # Create a header matching the OS' tcp-stack fingerprint.
        system_ua = None
        match platform.system():
            case 'Linux':
                system_ua = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'
            case 'Darwin':
                system_ua = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 15_7_2) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/26.0 Safari/605.1.15'
            case 'Windows':
                system_ua = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'
        try:
            request = requests.get(url, headers={'User-Agent': system_ua})
            if request.status_code != requests.codes.ok:
                print(f"Api.get() returned status {request.status_code}")
            return request.text
        except Exception as e:
            raise e

    class Limit(Enum):
        FREE = 0
        TIER_1 = 1
        TIER_2 = 2
        TIER_3 = 3
    
    class Model(Enum):
        GEN = 1
        EMB = 2
        LOC = 3

    class Const(Enum):
        STOP = "I don't know."
        METRIC_BATCH = 20
        SERIES_BATCH = 40
        EMBED_BATCH = 100
        CHUNK_MAX = 1500

        @classmethod
        def Stop(cls):
            return cls.STOP.value

        @classmethod
        def MetricBatch(cls):
            return cls.METRIC_BATCH.value

        @classmethod
        def SeriesBatch(cls):
            return cls.SERIES_BATCH.value

        @classmethod
        def EmbedBatch(cls):
            return cls.EMBED_BATCH.value

        @classmethod
        def ChunkMax(cls):
            return cls.CHUNK_MAX.value
    
    class Env(NamedTuple): # Make init args immutable.
        CLIENT: genai.Client
        API_LIMIT: int
        GEN_DEFAULT: str

    def __init__(self, with_limit: Limit | int, default_model: str):
        if default_model in self.gen_model.keys():
            self.write_lock = threading.RLock()
            try:
                if isinstance(with_limit, int) and with_limit in [id.value for id in Api.Limit]:
                    limit = with_limit
                else:
                    limit = with_limit.value
            except Exception as e:
                print(f"Api.__init__: {with_limit} is not a valid limit")
            else:
                self.args = Api.Env(
                    genai.Client(api_key=UserSecretsClient().get_secret("GOOGLE_API_KEY")),
                    limit, default_model)
            self.m_id = list(self.gen_model.keys()).index(default_model)
            self.default_model.append(default_model)
            self.update_quota()
            self.s_embed = GeminiEmbedFunction(self.args.CLIENT, semantic_mode = True) # type: ignore
            logging.getLogger("google_genai").setLevel(logging.WARNING) # suppress info on generate
        else:
            print(f"Api.__init__: {default_model} not found in gen_model.keys()")
        

    def __call__(self, model: Model) -> str:
        if model == self.Model.GEN:
            return "models/" + list(self.gen_model.keys())[self.m_id]
        elif model == self.Model.LOC:
            return self.gen_local[self.default_local]
        else:
            return "models/" + self.embed_model[0] if not self.embed_local else "embeddinggemma:latest"

    def push_default_model(self, model_id: str):
        if model_id in self.gen_model.keys():
            self.write_lock.acquire()
            self.stop_running()
            self.default_model.append(model_id)
            self.m_id = list(self.gen_model.keys()).index(model_id)
            self.write_lock.release()
        else:
            print(f"{model_id} not found in gen_model.keys()")

    def pop_default_model(self):
        if len(self.default_model) > 1:
            self.write_lock.acquire()
            self.stop_running()
            self.default_model.pop(-1)
            self.m_id = list(self.gen_model.keys()).index(self.default_model[-1])
            self.write_lock.release()

    def retriable(self, retry_fn: Callable, *args, **kwargs):
        tries = 3*len(self.gen_model.keys())
        for attempt in range(tries):
            try:
                self.write_lock.acquire()
                token_use = self.token_count(kwargs["contents"])
                if self.gen_rpm > self.min_rpm and token_use <= self.token_quota and self.token_quota > self.min_tpm:
                    self.token_quota -= token_use
                    self.gen_rpm -= 1
                else:
                    self.on_error(kwargs)
                if not self.running and not self.errored:
                    self.rpm_timer = Timer(self.dt_rpm, self.refill_rpm)
                    self.rpm_timer.start()
                    self.running = True
                return retry_fn(*args, **kwargs)
            except (errors.APIError, exceptions.RetryError) as api_error:
                if isinstance(api_error, errors.APIError):
                    is_retry = api_error.code in {429, 503, 500, 400} # code 400 when TPM exceeded
                    if api_error.code == 400:
                        print(f"retriable.api_error: token limit exceeded ({token_use})")
                    else:
                        print(f"retriable.api_error({api_error.code}): {str(api_error)}")
                    if not is_retry or attempt == tries:
                        raise api_error
                self.on_error(kwargs)
            except Exception as e:
                print(f"retriable.exception: {str(e)}")
                self.on_error(kwargs)
            finally:
                self.write_lock.release()

    def on_error(self, kwargs):
        self.generation_fail()
        kwargs["model"] = self(Api.Model.GEN)
        time.sleep(self.dt_between)

    def stop_running(self):
        if self.running:
            self.rpm_timer.cancel()
            self.running = False

    def validation_fail(self):
        list(self.gen_model.values())[self.m_id].err[0] += 1
        self.error_total += 1

    def generation_fail(self):
        self.stop_running()
        self.save_error()
        self.next_model()
        print("Api.generation_fail.next_model: model is now", list(self.gen_model.keys())[self.m_id])
        if not self.errored:
            self.error_timer = Timer(self.dt_err, self.zero_error)
            self.error_timer.start()
            self.errored = True

    def save_error(self):
        list(self.gen_model.values())[self.m_id].err[1] += 1
        self.error_total += 1

    def next_model(self):
        self.m_id = (self.m_id+1)%len(self.gen_model.keys())
        self.update_quota()

    def refill_rpm(self):
        self.running = False
        self.update_quota()
        print("Api.refill_rpm", self.gen_rpm)

    def zero_error(self):
        self.errored = False
        self.m_id = list(self.gen_model.keys()).index(self.default_model[-1])
        self.update_quota()
        print("Api.zero_error: model is now", list(self.gen_model.keys())[self.m_id])

    def update_quota(self):
        self.gen_rpm = list(self.gen_model.values())[self.m_id].rpm[self.args.API_LIMIT]
        self.token_quota = list(self.gen_model.values())[self.m_id].tpm[self.args.API_LIMIT]*1_000_000

    def token_count(self, expr: str | list):
        count = self.args.CLIENT.models.count_tokens(
            model=self(Api.Model.GEN),
            contents=json.dumps(expr) if isinstance(expr, str) else str(expr))
        return count.total_tokens

    def errors(self):
        errors = {"total": self.error_total, "by_model": {}}
        for m_code, m in self.gen_model.items():
            errors["by_model"].update({
                m_code: {
                    "api_related": m.err[1],
                    "validation": m.err[0]
                }})
        return errors

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def similarity(self, content: list):
        return self.s_embed.sts(content) # type: ignore

Skipping Laminar.initialize()


In [3]:
# Define the embedding function.
api = NewType("api", Api) # type: ignore (forward-decl)
class GeminiEmbedFunction:
    document_mode = True  # Generate embeddings for documents (T,F), or queries (F,F).
    semantic_mode = False # Semantic text similarity mode is exclusive (F,T).
    
    def __init__(self, genai_client, semantic_mode: bool = False):
        self.client = genai_client
        if semantic_mode:
            self.document_mode = False
            self.semantic_mode = True

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def __embed__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        elif not self.document_mode and not self.semantic_mode:
            embedding_task = "retrieval_query"
        elif not self.document_mode and self.semantic_mode:
            embedding_task = "semantic_similarity"
        partial = self.client.models.embed_content(
            model=api(Api.Model.EMB),
            contents=input,
            config=types.EmbedContentConfig(task_type=embedding_task)) # type: ignore
        return [e.values for e in partial.embeddings]
    
    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def __call__(self, input: Documents) -> Embeddings:
        try:
            response = []
            for i in range(0, len(input), Api.Const.EmbedBatch()):  # Gemini max-batch-size is 100.
                response += self.__embed__(input[i:i + Api.Const.EmbedBatch()])
            return response
        except Exception as e:
            print(f"caught exception of type {type(e)}\n{e}")
            raise e

    def sts(self, content: list) -> float:
        df = pandas.DataFrame(self(content), index=content)
        score = df @ df.T
        return score.iloc[0].iloc[1]

## Set Gemini API Limit

In [4]:
# Instantiate the api-helper with usage limit => FREE.
# Optional: Set limit here to one of [FREE,TIER_1,TIER_2,TIER_3]
api = Api(with_limit=Api.Limit.FREE, default_model="gemini-2.5-flash")
# Export api environment for agent.
os.environ["API_LIMIT"]=str(api.args.API_LIMIT)
os.environ["GEN_DEFAULT"]=api.args.GEN_DEFAULT
# Cleanup old vector_db instances.
!rm -rf vector_db

# Gemini Baseline Check

In [5]:
# This is an accurate retelling of events. 
config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
    temperature=0.0
)

chat = api.args.CLIENT.chats.create(
    model=api(Api.Model.GEN),
    config=config_with_search,
    history=[]) # Ignoring the part about dark elves, and tengwar.

response = chat.send_message('Do you know anything about the stock market?')
Markdown(response.text)

Yes, I can provide you with information about the stock market.

The stock market is a global network of exchanges and over-the-counter (OTC) marketplaces where investors buy and sell shares of publicly traded companies. These shares, also known as equities, represent fractional ownership in a company.

**Purpose and Function:**
The stock market serves two primary purposes:
1.  **Capital Raising for Companies:** It allows companies to raise capital (money) from the public by issuing shares, which they can then use to fund and expand their businesses. This process often begins with an Initial Public Offering (IPO), where a private company first lists its shares publicly on an exchange.
2.  **Investment Opportunities for Individuals:** It provides investors with the opportunity to share in the potential profits and growth of these companies. Investors can profit through dividends (regular payments from company profits) or by selling their shares at a higher price than they bought them for (capital gains).

**How the Stock Market Works:**
The stock market operates through a system of supply and demand.
*   **Primary Market:** This is where new stocks are first issued, typically through an IPO, allowing companies to sell shares directly to investors to raise capital.
*   **Secondary Market:** After the initial issuance, most daily trading occurs here. Investors buy and sell existing shares among themselves on stock exchanges like the New York Stock Exchange (NYSE) or Nasdaq. The company is not directly involved in these subsequent transactions.
*   **Price Determination:** Stock prices are primarily determined by supply and demand. If more buyers want a stock than sellers, the price tends to rise, and vice versa. This process is known as price discovery.

**Key Components and Participants:**
*   **Stock Exchanges:** These are organized and regulated platforms (often virtual) where stocks and other securities are bought and sold. They facilitate the exchange of securities and provide real-time trading information.
*   **Investors and Traders:** Participants range from small individual investors to large institutional investors like pension funds, mutual funds, insurance companies, and hedge funds.
*   **Brokers:** These intermediaries execute buy and sell orders on behalf of investors.
*   **Regulation:** In the United States, the Securities and Exchange Commission (SEC) regulates the stock market to protect investors and ensure fair practices.

**Types of Investments in the Stock Market:**
While "stock market" often refers to equities, it's part of a broader financial market where various instruments are traded:
*   **Stocks (Equities):** Represent ownership in a company.
    *   **Common Stock:** Gives shareholders voting rights on corporate decisions and the potential for higher returns, but also higher risk.
    *   **Preferred Stock:** Typically offers fixed dividend payments and a preference over common shareholders in receiving assets if the company liquidates, but usually no voting rights.
    *   **Categorization by Market Capitalization:** Stocks can also be classified by company size (large-cap, mid-cap, small-cap).
*   **Bonds:** When you buy a bond, you are essentially lending money to a government or corporation for a set period, receiving regular interest payments.
*   **Mutual Funds:** These are professionally managed portfolios that pool money from many investors to buy a diversified collection of stocks, bonds, or other securities.
*   **Exchange-Traded Funds (ETFs):** Similar to mutual funds, but they trade like individual stocks on exchanges throughout the day. Many ETFs track specific market indexes.
*   **Derivatives:** Financial contracts whose value is derived from an underlying asset, such as stocks, bonds, or commodities. They are complex and carry high risk.

**Factors Influencing Stock Prices:**
Stock prices are influenced by a variety of factors, including:
*   **Company Performance:** Earnings, profitability, and growth prospects are fundamental factors.
*   **Market Sentiment:** Investor psychology and overall market mood can cause short-term price changes.
*   **Economic Factors:** News, political events, and broader economic reports can impact stock prices.

In [6]:
response = chat.send_message('I have an interest in AMZN stock')
Markdown(response.text)

It's great you have an interest in AMZN stock! AMZN is the ticker symbol for **Amazon.com, Inc.**, one of the world's largest and most influential technology companies.

Here's some information about AMZN stock and the company:

**About Amazon (AMZN)**
Amazon is a multinational technology company focusing on e-commerce, cloud computing, digital streaming, and artificial intelligence. Its primary business segments include:
*   **Online Stores:** Its core e-commerce business, selling a vast array of products directly to consumers and through third-party sellers.
*   **Amazon Web Services (AWS):** A leading cloud computing platform providing on-demand cloud computing platforms and APIs to individuals, companies, and governments. This is a significant profit driver for Amazon.
*   **Advertising:** A rapidly growing segment that includes advertising services to sellers, vendors, publishers, authors, and others.
*   **Subscriptions:** Primarily Amazon Prime memberships, which offer benefits like free shipping, streaming video (Prime Video), and other services.
*   **Physical Stores:** Includes Whole Foods Market and Amazon Go stores.
*   **Other:** Includes devices like Kindle, Fire tablets, Echo, and other initiatives.

**Current Information (as of late 2025):**
To give you the most up-to-date information, I'll perform a search for its current stock price and recent news.

As of December 1, 2025, Amazon (AMZN) stock is trading around **$233.15 - $233.22 USD**.

Here are some key metrics and recent insights:
*   **Market Capitalization:** Amazon's market cap is approximately **$2.49 trillion USD**, making it one of the world's most valuable companies.
*   **Recent Performance:**
    *   The stock has fluctuated between $230.23 and $233.54 today.
    *   AMZN stock has risen by 7.80% compared to the previous week and 0.67% over the last month.
    *   Over the last year, Amazon.com, Inc. has shown a 12.68% increase.
    *   Its 52-week high was $258.60, reached on November 2, 2025, and its 52-week low was $161.38.
*   **P/E Ratio:** The price-to-earnings (P/E) ratio is around 32.95 - 32.96.
*   **Upcoming Earnings:** Amazon.com, Inc. is expected to release its next earnings report on January 29, 2026.

**Recent News and Factors Influencing AMZN:**
*   **AI Investments:** Amazon is heavily investing in artificial intelligence (AI), with plans to join major tech firms in a $400 billion investment in AI by 2025. The company is also pledging up to $50 billion to expand supercomputing capacity for U.S. government agencies, focusing on building advanced AWS data centers. These AI investments are seen as a significant driver for future growth.
*   **AWS Growth:** Amazon Web Services (AWS) continues to be a strong performer, with its acceleration leading the way in overall good results. However, AWS growth slowed to 17.5% year-over-year in Q2 2025, facing increased competition from Azure and Google Cloud.
*   **E-commerce and Competition:** While Amazon remains a leader in e-commerce, it faces increasing competition from ultra-low-cost business strategies employed by companies like Temu and Shein, which are capturing significant market shares.
*   **Regulatory Scrutiny:** Regulatory concerns are rising for large technology firms, including Amazon. Investigations by the EU in November 2025 could classify AWS as a "gatekeeper" under the Digital Markets Act (DMA), potentially leading to hefty fines and increased compliance costs. Amazon has also incurred a $2.5 billion FTC settlement over Prime subscription practices.
*   **Analyst Sentiment:** Analysts have a range of future price estimates for AMZN, with a maximum estimate of $360.00 USD and a minimum estimate of $250.00 USD. Some analyses suggest the stock might be undervalued at current levels, with an intrinsic value around $302.50 per share.

**Important Considerations for Investors:**
*   **Diversified Business Model:** Amazon's strength comes from its diversified business, with strong positions in e-commerce, cloud computing (AWS), advertising, and subscriptions.
*   **Growth vs. Value:** Amazon is often considered a growth stock, meaning investors expect its earnings and revenue to grow at an above-average rate.
*   **Volatility:** AMZN stock can be volatile, and it has experienced significant declines in the past.
*   **Macroeconomic Factors:** Broader economic conditions, consumer spending habits, and global supply chain issues can all impact Amazon's performance.

**Disclaimer:** I am an AI and cannot provide financial advice. Investing in the stock market carries risks, and it's crucial to conduct your own thorough research or consult with a qualified financial advisor before making any investment decisions.

In [7]:
response = chat.send_message('''Tell me about AMZN current share price, short-term trends, and bullish versus bearish predictions''')
Markdown(response.text)

As of late November and early December 2025, Amazon (AMZN) stock has shown recent upward movement, with analysts generally maintaining a bullish long-term outlook despite some short-term volatility and competitive pressures.

### AMZN Current Share Price

On Friday, November 28, 2025, Amazon's stock closed at approximately **$233.22 USD**. The stock has traded around this level as of December 1, 2025.

### Short-Term Trends

In the immediate short term, AMZN has experienced some positive momentum:
*   The stock increased by 1.79% on November 28, 2025.
*   It has risen 7.39% from a pivot bottom point identified on Thursday, November 20, 2025.
*   Over the past week, AMZN stock has risen by 7.80%, and over the last month, it has seen a 0.67% increase.
*   Technical indicators suggest a generally bullish sentiment, with 23 technical analysis indicators signaling bullish signals against 3 bearish signals as of November 30, 2025.
*   The stock is currently in a "wide and weak rising trend" in the short term and is projected to rise by 2.58% over the next three months, with a 90% probability of trading between $223.26 and $262.43 by the end of this period.
*   A "Golden Cross," a bullish signal where the short-term moving average crosses above the long-term moving average, indicates potential upward momentum. However, some analysis also notes a "general sell signal" from the relationship where the long-term average is above the short-term average.
*   A potential cautionary sign is that volume decreased on the last trading day despite price gains, which could indicate a divergence between volume and price.

### Bullish Versus Bearish Predictions

**Bullish Predictions:**
*   **Strong Analyst Consensus:** The overwhelming majority of analysts rate AMZN as a "Strong Buy" or "Moderate Buy." Out of 44 analysts, 43 have a "Buy" rating and 1 has a "Hold" rating.
*   **High Price Targets:** The average 12-month price target from analysts ranges from $280.47 to $295.23, suggesting an upside potential of 20.26% to 28.83% from current levels. The highest price targets reach $335 to $340.
*   **Long-Term Growth Drivers:** Analysts anticipate continued strong growth from Amazon Web Services (AWS), which is expected to maintain its lead in cloud infrastructure services, and significant gains from Amazon's investments in artificial intelligence (AI). The company's e-commerce business is also expected to become more profitable due to cost-cutting and logistics improvements.
*   **Attractive Valuation:** Despite its size, Amazon's valuation is considered "enticing" by some, with shares trading at a forward price-to-earnings ratio of 29.
*   **Economic Moat:** Amazon possesses a wide economic moat, characterized by its network effect in online marketplaces, switching costs for AWS customers, massive scale, and strong brand, making it difficult to disrupt.
*   **Forecasted Price Increases:** Some forecasts predict AMZN could reach an average of $235 in December 2025, with a maximum of $270. Longer-term, some estimates project the stock price to reach $524.67 per share by the end of 2030.

**Bearish/Cautious Predictions:**
*   **Short-Term Pullback:** Some short-term forecasts suggest a potential slight decline, with one prediction indicating a drop of -3.41% to $225.23 by December 29, 2025. Another forecast suggests an end-of-month price of $228 for December 2025, representing a -6.6% change for the month.
*   **Fear & Greed Index:** The Fear & Greed Index is currently showing "Fear" at 39, which can indicate investor apprehension.
*   **Competition in Cloud Services:** While AWS remains strong, its growth has slowed compared to competitors like Microsoft Azure and Google Cloud, raising concerns about potential market share losses.
*   **Regulatory Scrutiny:** Amazon faces increasing regulatory scrutiny, including potential classification of AWS as a "gatekeeper" under the EU's Digital Markets Act, which could lead to fines and higher compliance costs. (This information was from a previous turn, but remains a relevant cautionary factor).
*   **E-commerce Competition:** The e-commerce segment faces growing competition from ultra-low-cost business models like Temu and Shein. (This information was from a previous turn, but remains a relevant cautionary factor).

In [8]:
response = chat.send_message('''What is mgm studio's stock ticker symbol?''')
Markdown(response.text)

MGM Studios, also known as Metro-Goldwyn-Mayer Studios Inc., does not have its own stock ticker symbol because it is no longer a publicly traded company. It was acquired by Amazon in 2022 and is now a wholly owned subsidiary, operating under the umbrella of Amazon MGM Studios.

Therefore, the ownership of MGM Studios is indirectly held by Amazon's shareholders. If you are interested in investing in the company that owns MGM Studios, you would look at **Amazon.com, Inc.**, which trades under the ticker symbol **AMZN** on the Nasdaq exchange.

It is important not to confuse MGM Studios with **MGM Resorts International**, which is a separate, publicly traded hospitality and casino entertainment company with the stock ticker symbol **MGM** on the New York Stock Exchange (NYSE).

In [9]:
response = chat.send_message('''What was the last open,close,high,low data for AMZN again?''')
Markdown(response.text)

Here is the last available open, close, high, and low data for AMZN, specifically for **Friday, November 28, 2025**:

*   **Close/Last Price:** $233.22
*   **Open Price:** $231.24
*   **High Price:** $233.28
*   **Low Price:** $230.22

In [10]:
response = chat.send_message('''What is AMZN open,close,high,low data for the past month? 
Present the data with multiple columns for display in markdown.''')
Markdown(response.text)

Here is the open, close, high, and low data for AMZN for the past month, covering November 2025 and the last few trading days of October 2025:

| Date        | Open (USD) | High (USD) | Low (USD) | Close (USD) |
| :---------- | :--------- | :--------- | :-------- | :---------- |
| Nov 28, 2025 | $231.24    | $233.285   | $230.22   | $233.22     |
| Nov 26, 2025 | $230.74    | $231.7474  | $228.77   | $229.16     |
| Nov 25, 2025 | $226.38    | $230.52    | $223.80   | $229.67     |
| Nov 24, 2025 | $222.555   | $227.33    | $222.27   | $226.28     |
| Nov 21, 2025 | $216.345   | $222.21    | $215.18   | $220.69     |
| Nov 20, 2025 | $227.05    | $227.41    | $216.74   | $217.14     |
| Nov 19, 2025 | $223.735   | $223.735   | $218.52   | $222.69     |
| Nov 18, 2025 | $228.10    | $230.20    | $222.42   | $222.55     |
| Nov 17, 2025 | $233.25    | $234.60    | $229.19   | $232.87     |
| Nov 14, 2025 | $235.06    | $238.73    | $232.89   | $234.69     |
| Nov 13, 2025 | $243.05    | $243.75    | $236.50   | $237.58     |
| Nov 12, 2025 | $250.24    | $250.37    | $243.75   | $244.25     |
| Nov 11, 2025 | $248.41    | $249.75    | $247.23   | $249.10     |
| Nov 10, 2025 | $248.34    | $251.75    | $245.59   | $248.40     |
| Nov 07, 2025 | $242.90    | $244.90    | $238.49   | $244.41     |
| Nov 06, 2025 | $249.16    | $250.38    | $242.17   | $243.04     |
| Nov 05, 2025 | $249.03    | $251.00    | $246.16   | $250.20     |
| Nov 04, 2025 | $250.38    | $257.01    | $248.66   | $249.32     |
| Nov 03, 2025 | $255.36    | $258.60    | $252.90   | $254.00     |
| Oct 31, 2025 | $250.10    | $250.50    | $243.98   | $244.22     |
| Oct 30, 2025 | $227.06    | $228.44    | $222.75   | $222.86     |
| Oct 29, 2025 | $231.67    | $232.82    | $227.76   | $230.30     |
| Oct 28, 2025 | $228.22    | $231.49    | $226.21   | $229.25     |
| Oct 27, 2025 | $227.66    | $228.40    | $225.54   | $226.97     |
| Oct 24, 2025 | $221.97    | $225.40    | $221.90   | $224.21     |

Please note that stock market data can vary slightly across different sources due to reporting times or adjustments. The data above is compiled from available historical information for the specified period.

# Previously on Kaggle: StockChat 1.0

## Validation BaseModels

In [11]:
# Validation BaseModels in pydantic schema.
class RestStatus(Enum):
    OK = "OK"
    DELAY = "DELAYED"
    NONE = "NOT_FOUND"
    AUTH = "NOT_AUTHORIZED"

class StopGeneration(BaseModel):
    result: str = Api.Const.Stop()

class RestResultPoly(BaseModel):
    request_id: Optional[str] = None
    count: Optional[int] = None
    next_url: Optional[str] = None
    status: RestStatus  

class MarketSession(Enum):
    PRE = "pre-market"
    REG = "regular"
    POST = "post-market"
    CLOSED = "closed"
    NA = "not applicable"

class MarketEvent(Enum):
    PRE_OPEN = 0
    REG_OPEN = 1
    REG_CLOSE = 2
    POST_CLOSE = 3
    LAST_CLOSE = 4

class AssetClass(Enum):
    STOCKS = "stocks"
    OPTION = "options"
    CRYPTO = "crypto"
    FOREX = "fx"
    INDEX = "indices"
    OTC = "otc"

class SymbolType(Enum):
    COMMON = "Common Stock"
    ETP = "ETP"
    ADR = "ADR"
    REIT = "REIT"
    DELISTED = ""
    CEF = "Closed-End Fund"
    UNIT = "Unit"
    RIGHT = "Right"
    EQUITY = "Equity WRT"
    GDR = "GDR"
    PREF = "Preference"
    CDI = "CDI"
    NVDR = "NVDR"
    REG = "NY Reg Shrs"
    MLP = "MLP"
    MUTUAL = "Mutual Fund"

class Locale(Enum):
    US = "us"
    GLOBAL = "global"

class Sentiment(Enum):
    V_POS = "very positive"
    POSITIVE = "positive"
    NEUTRAL_P = "neutral/positive"
    NEUTRAL_SP = "neutral/slightly positive"
    NEUTRAL = "neutral"
    NEUTRAL_SN = "neutral/slightly negative"
    NEUTRAL_N = "neutral/negative"
    MIXED = "mixed"
    NEGATIVE = "negative"
    V_NEG = "very negative"

class Trend(Enum):
    S_BUY = "strong-buy"
    BUY = "buy"
    HOLD = "hold"
    SELL = "sell"
    S_SELL = "strong-sell"

class MarketCondition(Enum):
    BULL = "bullish"
    BULLN = "cautiously bullish"
    HOLD = "hold"
    BEARN = "cautiously bearish"
    BEAR = "bearish"

class GeneratedEvent(BaseModel):
    last_close: str
    pre_open: str
    reg_open: str
    reg_close: str
    post_close: str
    timestamp: Optional[str] = None
    is_holiday: Optional[bool] = None

    def model_post_init(self, *args, **kwargs) -> None:
        if self.timestamp is None:
            self.timestamp = datetime.now(self.tz()).strftime('%c')
        if self.is_holiday is None:
            self.is_holiday = False

    def session(self, with_date: Optional[str] = None) -> MarketSession:
        if with_date is None:
            with_date = datetime.now(self.tz()).strftime('%c')
        compare = parse(with_date)
        if self.is_holiday or compare.weekday() > 4: # weekend
            return MarketSession.CLOSED
        events = [parse(event).time() for event in [self.pre_open,self.reg_open,self.reg_close,self.post_close]]
        if compare.time() < events[0]:
            return MarketSession.CLOSED
        else:
            session = MarketSession.NA
            if compare.time() >= events[0]:
                session = MarketSession.PRE
            if compare.time() >= events[1]:
                session = MarketSession.REG
            if compare.time() >= events[2]:
                session = MarketSession.POST
            if compare.time() >= events[3]:
                session = MarketSession.CLOSED
        return session

    def is_open(self) -> bool:
        return self.session() != MarketSession.CLOSED

    def has_update(self) -> bool:
        datetime_now = datetime.now(self.tz())
        self_ts = parse(self.timestamp)
        # Re-generate events for a new day.
        if datetime_now.day > self_ts.day:
            return True
        # No updates on holidays or when generated after post_close.
        if self.is_holiday or self_ts.time() >= parse(self.post_close).time():
            return False
        # Compare current time to generated event times.
        for event in [self.pre_open,self.reg_open,self.reg_close]:
            if datetime_now.time() > parse(event).time():
                return True
        # Current time is before pre_open.
        return False

    @classmethod
    def tz(cls):
        return pytz.timezone('US/Eastern') # Exchanges data is in eastern time.
    
    @classmethod
    def apply_fix(cls, value, fix: datetime) -> tuple[str, datetime]:
        api.validation_fail()
        value = fix.strftime('%c')
        return value, fix
    
    @field_validator("last_close")
    def valid_close(cls, value):
        date_gen = parse(value) # Generated close is in eastern time and tzinfo naive.
        date_now = parse(datetime.now(cls.tz()).strftime('%c')) # Need now in same format as generated.
        # Soft-pass: when actual session is closed after post-market
        if date_now.day == date_gen.day+1 and date_now.weekday() <= 4:
            date_fix = date_gen.replace(day=date_now.day)
            if date_fix.timestamp() < date_now.timestamp():
                value, date_gen = cls.apply_fix(value, date_fix) # soft-pass: use today's close
        # Soft-pass: when actual session is open post-market
        if date_now.day == date_gen.day and date_now.timestamp() < date_gen.timestamp():
            if date_now.weekday() > 0:
                date_fix = date_gen.replace(day=date_now.day-1)
            else:
                date_fix = date_gen.replace(day=date_now.day-3)
            if date_now.timestamp() > date_fix.timestamp():
                value, date_gen = cls.apply_fix(value, date_fix) # soft-pass: use previous close
        if date_now.weekday() == 0 or date_now.weekday() == 1 and date_gen.weekday() <= 4: # 0=monday, 4=friday
            return value # pass: generated thurs/friday on a monday/tues
        elif date_now.weekday() > 0 and date_now.weekday() <= 4 and date_gen.weekday() <= date_now.weekday()-1:
            return value # pass: generated yesterday/prior on a tues-fri
        elif date_now.weekday() > 4 and date_gen.weekday() <= 4:
            return value # pass: generated thurs/friday on a weekend
        elif date_now.day == date_gen.day and date_now.timestamp() > date_gen.timestamp():
            return value # pass: generated today after closed
        elif date_now.timestamp() < date_gen.timestamp():
            raise ValueError("last close cannot be a future value")
        else:
            raise ValueError("generated invalid last close")
        api.validation_fail()

class VectorStoreResult(BaseModel):
    docs: str
    dist: Optional[float] # requires query
    meta: Optional[dict]  # requires get or query
    store_id: str

class Aggregate(RestResultPoly):
    symbol: str
    open: float
    high: float
    low: float
    close: float
    volume: int
    otc: Optional[bool] = None
    preMarket: Optional[float] = None
    afterHours: Optional[float] = None

class DailyCandle(Aggregate):
    from_date: str

class AggregateWindow(BaseModel):
    o: float
    h: float
    l: float
    c: float
    v: int # traded volume
    n: Optional[int] = None # transaction count
    vw: Optional[float] = None # volume weighted average price
    otc: Optional[bool] = None
    t: int

    @field_validator("t")
    def valid_t(cls, value):
        if not value > 0:
            raise ValueError("invalid timestamp")
        if len(str(value)) == 13:
            return int(value/1000)
        return value

class CustomCandle(RestResultPoly): 
    ticker: str
    adjusted: bool
    queryCount: int
    resultsCount: int
    results: list[AggregateWindow]

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.results)

    def get(self) -> list[AggregateWindow]:
        return self.results
    
class MarketStatus(BaseModel):
    exchange: str
    holiday: Optional[str] = None
    isOpen: bool
    session: Optional[MarketSession] = None
    t: int
    timezone: str

    def model_post_init(self, *args, **kwargs) -> None:
        if self.session is None:
            self.session = MarketSession.CLOSED
        if self.holiday is None:
            self.holiday = MarketSession.NA.value

class MarketStatusResult(BaseModel):
    results: MarketStatus

    def get(self) -> MarketStatus:
        return self.results

class Symbol(BaseModel):
    description: str
    displaySymbol: str
    symbol: str
    type: SymbolType

class SymbolResult(BaseModel):
    count: int
    result: list[Symbol]

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.result)

    def get(self) -> list[Symbol]:
        return self.result

class Quote(BaseModel):
    c: float
    d: float
    dp: float
    h: float
    l: float
    o: float
    pc: float
    t: int

    @field_validator("t")
    def valid_t(cls, value):
        if not value > 0:
            raise ValueError("invalid timestamp")
        return value

class PeersResult(BaseModel):
    results: list[str]
    count: Optional[int] = None

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.results)

    def get(self) -> list[str]:
        return self.results

class BasicFinancials(BaseModel):
    metric: dict
    metricType: str
    series: dict
    symbol: str

class Insight(BaseModel):
    sentiment: Sentiment|MarketCondition
    sentiment_reasoning: str
    ticker: str

class Publisher(BaseModel):
    favicon_url: Optional[str]
    homepage_url: str
    logo_url: str
    name: str

class NewsSummary(BaseModel):
    title: str
    summary: Optional[str]
    insights: Optional[list[Insight]]
    published_utc: str

class NewsTypePoly(BaseModel):
    amp_url: Optional[str] = None
    article_url: str
    title: str
    author: str
    description: Optional[str] = None
    id: str
    image_url: Optional[str] = None
    insights: Optional[list[Insight]] = None
    keywords: Optional[list[str]] = None
    published_utc: str
    publisher: Publisher
    tickers: list[str]

    def summary(self):
        return NewsSummary(title=self.title,
                           summary=self.description,
                           insights=self.insights,
                           published_utc=self.published_utc)

class NewsResultPoly(RestResultPoly):
    results: list[NewsTypePoly]

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.results)

    def get(self) -> list[NewsTypePoly]:
        return self.results

class NewsTypeFinn(BaseModel):
    category: str
    datetime: int
    headline: str
    id: int
    image: str
    related: str # symbol
    source: str
    summary: str
    url: str

    def summary(self):
        return NewsSummary(title=self.headline,
                           summary=self.summary,
                           insights=None,
                           published_utc=self.datetime)

class NewsResultFinn(BaseModel):
    results: list[NewsTypeFinn]
    count: Optional[int] = None

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.results)

    def get(self) -> list[NewsTypeFinn]:
        return self.results

class NewsTypeGenerated(BaseModel):
    title: str
    summary: str
    insights: list[Insight]
    keywords: list[str]
    source: Publisher
    published_utc: str
    tickers: list[str]
    url: str

    def summary(self):
        return NewsSummary(title=self.title,
                           summary=self.summary,
                           insights=self.insights,
                           published_utc=self.published_utc)

class TickerOverview(BaseModel):
    ticker: str
    name: str
    market: AssetClass
    locale: Locale
    primary_exchange: Optional[str] = None
    active: bool
    currency_name: str
    cik: Optional[str] = None
    composite_figi: Optional[str] = None
    share_class_figi: Optional[str] = None
    market_cap: Optional[int|float] = None
    phone_number: Optional[str] = None
    address: Optional[dict] = None
    description: Optional[str] = None
    sic_code: Optional[str] = None
    sic_description: Optional[str] = None
    ticker_root: Optional[str] = None
    homepage_url: Optional[str] = None
    total_employees: Optional[int] = None
    list_date: Optional[str] = None
    branding: Optional[dict] = None
    share_class_shares_outstanding: Optional[int] = None
    weighted_shares_outstanding: Optional[int] = None
    round_lot: Optional[int] = None

class OverviewResult(RestResultPoly):
    results: TickerOverview

    def get(self) -> TickerOverview:
        return self.results

class RecommendationTrend(BaseModel):
    buy: int
    hold: int
    period: str
    sell: int
    strongBuy: int
    strongSell: int
    symbol: str

class TrendsResult(BaseModel):
    results: list[RecommendationTrend]
    count: Optional[int] = None

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.results)

    def get(self) -> list[RecommendationTrend]:
        return self.results

## Contents Memory

In [12]:
# A contents-memory object.
class Memory:
    def __init__(self):
        self.system = f"""Give a concise, and detailed summary. Use information that you learn from the API responses.
        Use your tools and function calls according to the rules. Convert any all-upper case identifiers
        to proper case in your response. Convert any abbreviated or shortened identifiers to their full forms.
        Convert timestamps according to the rules before including them. Think step by step.
        """
        self.revery = {}
        self.contents = []
        self.prompt = None
        self.summary = None
        self.response = None
    
    def set_prompt(self, prompt):
        self.prompt = f"""
        The current date and time is: {datetime.now(GeneratedEvent.tz()).strftime('%c')}
        
        {prompt}
        """
        self.contents = [types.Content(role="user", parts=[types.Part(text=self.prompt)])]

    def set_reason(self, step):
        # Append the model's reasoning part.
        self.contents.append(types.Content(role="model", parts=[types.Part(thought=True,text=step)]))

    def append_code(self, prompt, code_response_parts):
        subroutine_content = [types.Content(role="user", parts=[types.Part(text=prompt)]),
                              types.Content(role="model", parts=code_response_parts)]
        # Append the model's generated code and execution result.
        self.revery[datetime.now(GeneratedEvent.tz()).strftime('%c')] = { 
            "contents": subroutine_content
        }

    def update_contents(self, function_call, api_response_part):
        # Append the model's function call part.
        self.contents.append(types.Content(role="model", parts=[types.Part(function_call=function_call)])) 
        # Append the api response part.
        self.contents.append(types.Content(role="user", parts=[api_response_part]))

    def set_summary(self, summary):
        self.summary = summary
        self.contents.append(types.Content(role="model", parts=[types.Part(text=summary)]))
        self.revery[datetime.now(GeneratedEvent.tz()).strftime('%c')] = {
            "prompt": self.prompt, 
            "summary": self.summary, 
            "contents": self.contents
        }
        self.contents = []

memory = Memory()

## Retrieval-Augmented Generation

In [13]:
# Define tool: retrieval-augmented generation.
# - using Chroma and text-embedding-004 for storage and retrieval
# - using gemini-2.0-flash for augmented generation
class RetrievalAugmentedGenerator:
    chroma_client = chromadb.PersistentClient(path="vector_db")
    config_temp = types.GenerateContentConfig(temperature=0.0)
    exchange_codes: Optional[dict] = None
    exchange_lists: dict = {}
    events: dict = {}
    holidays: dict = {}

    def __init__(self, genai_client, collection_name):
        self.client = genai_client
        self.embed_fn = GeminiEmbedFunction(genai_client)
        self.db = self.chroma_client.get_or_create_collection(
            name=collection_name, 
            embedding_function=self.embed_fn,  # type: ignore
            metadata={"hnsw:space": "cosine"})
        logging.getLogger("chromadb").setLevel(logging.ERROR) # suppress warning on existing id
        self.set_holidays("US", ["09-01-2025","10-13-2025","11-11-2025","11-27-2025","12-25-2025"])
        #self.generated_events("US")

    def set_holidays(self, exchange_code: str, holidays: list):
        self.holidays[exchange_code] = [datetime.strptime(h, "%m-%d-%Y").date() for h in holidays]

    def get_exchange_codes(self, with_query: Optional[str] = None):
        gen = None
        if with_query and with_query not in self.exchange_lists.keys():
            gen = tqdm(total=1, desc="Generate exchange codes with_query")
            data = self.get_exchanges_csv(
                f"""What is the {with_query} exchange code? Return only the exchange codes 
                as a list in string form. Just the list string. 
                Omit all other information or details. Do not chat or use sentences.""").candidates[0].content
            self.exchange_lists[with_query] = ast.literal_eval(data.parts[-1].text)
        elif with_query is None and self.exchange_codes is None:
            gen = tqdm(total=1, desc="Generate exchange codes")
            data = self.get_exchanges_csv(
                """Give me a dictionary in string form. It must contain key:value pairs 
                mapping exchange code to name. Just the dictionary string. 
                Omit all other information or details. Do not chat or use sentences.""").candidates[0].content
            self.exchange_codes = ast.literal_eval(data.parts[-1].text.strip("\\`"))
        if gen:
            gen.update(1)
        return self.exchange_lists[with_query] if with_query else self.exchange_codes

    def get_event_date(self, event_t: str, exchange_code: str, event: MarketEvent):
        current_dt_str = datetime.now(GeneratedEvent.tz()).strftime('%c')
        current_dt = datetime.strptime(current_dt_str, "%a %b %d %H:%M:%S %Y")
        current_t_str = datetime.now(GeneratedEvent.tz()).strftime('%H:%M:%S')
        current_t = datetime.strptime(current_t_str, "%H:%M:%S").time()
        event_time = parse(event_t).time()
        gen_datetime = None
        if event is MarketEvent.LAST_CLOSE:
            last_close_day = current_dt.date() - timedelta(days=0 if current_t > event_time else 1)
            # Loop backwards to find the last valid trading day (not a weekend or holiday).
            while last_close_day.weekday() >= 5 or last_close_day in self.holidays[exchange_code]: # 5 = Sat, 6 = Sun
                last_close_day -= timedelta(days=1)
            # Combine the date and time.
            gen_datetime = datetime.combine(last_close_day, event_time)
        else:
            next_event_day = current_dt.date() + timedelta(days=0 if current_t < event_time else 1)
            # Loop forward to find the next valid trading day (not a weekend or holiday).
            while next_event_day.weekday() >= 5 or next_event_day in self.holidays[exchange_code]: # 5 = Sat, 6 = Sun
                next_event_day += timedelta(days=1)
            # Combine date and time.
            gen_datetime = datetime.combine(next_event_day, event_time)
        # Format the result as requested.
        return gen_datetime.strftime('%a %b %d %X %Y')

    def generate_event(self, exchange_code: str, event: MarketEvent):
        if event is MarketEvent.LAST_CLOSE or event is MarketEvent.POST_CLOSE:
            prompt = f"""What is the closing time including post_market hours."""
        elif event is MarketEvent.PRE_OPEN or event is MarketEvent.REG_OPEN:
            is_pre = "including" if event is MarketEvent.PRE_OPEN else "excluding"
            prompt = f"""What is the opening time {is_pre} pre_market hours."""
        elif event is MarketEvent.REG_CLOSE:
            prompt = f"""What is the closing time excluding post_market hours."""
        prompt = f"""Answer based on your knowledge of exchange operating hours.
            Do not answer in full sentences. Omit all chat and provide the answer only.
            The fields pre_market and post_market both represent extended operating hours.

            The current date and time: {datetime.now(GeneratedEvent.tz()).strftime('%c')}
            
            Consider the {exchange_code} exchange's operating hours.
            {prompt}
            
            Answer with the time in this format: '%H:%M:%S'.
            Omit all other chat and details. Do not use sentences."""
        progress = tqdm(total=1, desc=f"Generate {exchange_code}->{event}")
        response = self.get_exchanges_csv(prompt).candidates[0].content
        try:
            if Api.Const.Stop() in f"{response.parts[-1].text}":
                self.generate_event_failed(progress, exchange_code, event)
            else:
                response = self.get_event_date(response.parts[-1].text, exchange_code, event)
                progress.update(1)
                return response
        except Exception as e:
            self.generate_event_failed(progress, exchange_code, event)

    def generate_event_failed(self, progress: tqdm, exchange_code: str, event: MarketEvent):
        progress.close()
        api.generation_fail()
        time.sleep(api.dt_between)
        return self.generate_event(exchange_code, event)

    def generated_events(self, exchange_code: str) -> GeneratedEvent:
        # Check for an existing GeneratedEvent object having updates.
        if exchange_code in self.events.keys() and self.events[exchange_code].has_update():
            event_obj = self.events[exchange_code]
            event_state = [(event_obj.pre_open, MarketEvent.PRE_OPEN),
                           (event_obj.reg_open, MarketEvent.REG_OPEN),
                           (event_obj.reg_close, MarketEvent.REG_CLOSE),
                           (event_obj.post_close, MarketEvent.POST_CLOSE)]
            # Need now in same format as generated.
            datetime_now = parse(datetime.now(event_obj.tz()).strftime('%c'))
            gen_ts = parse(event_obj.timestamp)
            # Re-generate events when day changes.
            if datetime_now.day > gen_ts.day:
                del self.events[exchange_code]
                return self.generated_events(exchange_code)
            # Update changed events on trading days.
            for e in event_state:
                if datetime_now > parse(e[0]):
                    event_dt = self.generate_event(exchange_code, e[1])
                    match e[1]:
                        case MarketEvent.PRE_OPEN:
                            event_obj.pre_open = event_dt
                        case MarketEvent.REG_OPEN:
                            event_obj.reg_open = event_dt
                        case MarketEvent.REG_CLOSE:
                            event_obj.reg_close = event_dt
                        case MarketEvent.POST_CLOSE:
                            event_obj.post_close = event_dt
            event_obj.timestamp = datetime.now(event_obj.tz()).strftime('%c')
            self.events[exchange_code] = event_obj
        # Generate events for an exchange code not in cache.
        elif exchange_code not in self.events.keys():
            self.events[exchange_code] = GeneratedEvent(
                last_close=self.generate_event(exchange_code, MarketEvent.LAST_CLOSE),
                pre_open=self.generate_event(exchange_code, MarketEvent.PRE_OPEN),
                reg_open=self.generate_event(exchange_code, MarketEvent.REG_OPEN),
                reg_close=self.generate_event(exchange_code, MarketEvent.REG_CLOSE),
                post_close=self.generate_event(exchange_code, MarketEvent.POST_CLOSE),
                is_holiday=datetime.now().date() in self.holidays[exchange_code])
        return self.events[exchange_code]

    def set_holiday_event(self, exchange_code: str):
        self.generated_events(exchange_code).is_holiday = True

    def last_market_close(self, exchange_code: str):
        return self.generated_events(exchange_code).last_close

    def add_documents_list(self, docs: list):
        self.embed_fn.document_mode = True # Switch to document mode.
        ids = list(map(str, range(self.db.count(), self.db.count()+len(docs))))
        metas=[{"source": doc.metadata["source"]} for doc in docs]
        content=[doc.page_content for doc in docs]
        tqdm(self.db.add(ids=ids, documents=content, metadatas=metas), desc="Generate document embedding")

    def add_api_document(self, query: str, api_response: str, topic: str, source: str = "add_api_document"):
        self.embed_fn.document_mode = True # Switch to document mode.
        splitter = RecursiveJsonSplitter(max_chunk_size=Api.Const.ChunkMax())
        docs = splitter.create_documents(texts=[api_response], convert_lists=True)
        ids = list(map(str, range(self.db.count(), self.db.count()+len(docs))))
        content = [json.dumps(doc.page_content) for doc in docs]
        metas = [{"source": source, "topic": topic}]*len(docs)
        tqdm(self.db.add(ids=ids, documents=content, metadatas=metas), desc="Generate api embedding")

    def add_peers_document(self, query: str, names: list, topic: str, source: str, group: str):
        self.embed_fn.document_mode = True # Switch to document mode.
        peers = {"symbol": topic, "peers": names}
        tqdm(self.db.add(ids=str(self.db.count()),
                         documents=[json.dumps(peers)],
                         metadatas=[{"source": source, "topic": topic, "group": group}]),
             desc="Generate peers embedding")

    def get_peers_document(self, query: str, topic: str, group: str):
        return self.get_documents_list(query, where={"$and": [{"group": group}, {"topic": topic}]})

    def add_rest_chunks(self, chunks: list, topic: str, source: str, ids: Optional[list[str]] = None,
                        meta_opt: Optional[list[dict]] = None, is_update: bool = True):
        self.embed_fn.document_mode = True # Switch to document mode
        if ids is None:
            ids = list(map(str, range(self.db.count(), self.db.count()+len(chunks))))
        if isinstance(chunks[0], BaseModel):
            docs = [model.model_dump_json() for model in chunks]
        else:
            docs = [json.dumps(obj) for obj in chunks]
        meta_base = {"source": source, "topic": topic}
        if meta_opt is not None:
            for m in meta_opt:
                m.update(meta_base)
        metas = [meta_base]*len(chunks) if meta_opt is None else meta_opt
        if is_update:
            tqdm(self.db.upsert(ids=ids, documents=docs, metadatas=metas), desc="Upsert chunks embedding")
        else:
            tqdm(self.db.add(ids=ids, documents=docs, metadatas=metas), desc="Add chunks embedding")

    def get_market_status(self, exchange_code: str) -> tuple[list[VectorStoreResult], bool]: # result, has rest update
        self.embed_fn.document_mode = False # Switch to query mode.
        stored = self.stored_result(self.db.get(where={
            "$and": [{"exchange": exchange_code}, {"topic": "market_status"}]}))
        if len(stored) == 0:
            return stored, True
        # Check for a daily market status update.
        status = json.loads(stored[0].docs)
        gen_day = parse(self.generated_events(exchange_code).timestamp).day
        store_day = parse(stored[0].meta['timestamp']).day
        if status["holiday"] != MarketSession.NA.value and gen_day == store_day:
            return stored, False
        elif gen_day > store_day:
            return stored, True
        # Update with generated events to avoid rest api requests.
        status["session"] = self.generated_events(exchange_code).session().value
        status["isOpen"] = self.generated_events(exchange_code).is_open()
        stored[0].docs = json.dumps(status)
        return stored, False

    def get_basic_financials(self, query: str, topic: str, source: str = "get_financials_1"):
        return self.get_documents_list(
            query, max_sources=200, where={"$and": [{"source": source}, {"topic": topic}]})

    def add_quote_document(self, query: str, quote: str, topic: str, timestamp: int, source: str):
        self.embed_fn.document_mode = True # Switch to document mode.
        tqdm(self.db.add(ids=str(self.db.count()), 
                             documents=[quote], 
                             metadatas=[{"source": source, "topic": topic, "timestamp": timestamp}]), 
             desc="Generate quote embedding")

    def get_api_documents(self, query: str, topic: str, source: str = "add_api_document", 
                          meta_opt: Optional[list[dict]] = None):
        where = [{"source": source}, {"topic": topic}]
        if meta_opt is None:
            return self.get_documents_list(query, where={"$and": where})
        else:
            for meta in meta_opt:
                for k,v in meta.items():
                    where.append({k: v})
            return self.get_documents_list(query, where={"$and": where})

    def query_api_documents(self, query: str, topic: str, source: str = "add_api_document"):
        return self.generate_answer(query, where={"$and": [{"source": source}, {"topic": topic}]})

    def add_grounded_document(self, query: str, topic: str, result):
        self.embed_fn.document_mode = True # Switch to document mode.
        chunks = result.candidates[0].grounding_metadata.grounding_chunks
        supports = result.candidates[0].grounding_metadata.grounding_supports
        if supports is not None: # Only add grounded documents which have supports
            grounded_text = [f"{s.segment.text}" for s in supports]
            source = [f"{c.web.title}" for c in chunks]
            score = [f"{s.confidence_scores}" for s in supports]
            tqdm(self.db.add(ids=str(self.db.count()),
                             documents=json.dumps(grounded_text),
                             metadatas=[{"source": ", ".join(source),
                                         "confidence_score": ", ".join(score),
                                         "topic": topic,
                                         "question": query}]),
                 desc="Generate grounding embedding")

    def get_grounding_documents(self, query: str, topic: str):
        self.embed_fn.document_mode = False # Switch to query mode.
        return self.stored_result(self.db.get(where={"$and": [{"question": query}, {"topic": topic}]}))
            
    def add_wiki_documents(self, title: str, wiki_chunks: list):
        self.embed_fn.document_mode = True # Switch to document mode.
        result = self.get_wiki_documents(title)
        if len(result) == 0:
            ids = list(map(str, range(self.db.count(), self.db.count()+len(wiki_chunks))))
            metas=[{"title": title, "source": "add_wiki_documents"}]*len(wiki_chunks)
            tqdm(self.db.add(ids=ids, documents=wiki_chunks, metadatas=metas), desc="Generate wiki embeddings")

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def generate_with_wiki_passages(self, query: str, title: str, passages: list[str]):
        return self.generate_answer(query, where={"title": title}, passages=passages)
    
    def get_wiki_documents(self, title: Optional[str] = None):
        self.embed_fn.document_mode = False # Switch to query mode.
        if title is None:
            return self.stored_result(self.db.get(where={"source": "add_wiki_document"}))
        else:
            return self.stored_result(self.db.get(where={"title": title}))

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_documents_list(self, query: str, max_sources: int = 5000, where: Optional[dict] = None):
        self.embed_fn.document_mode = False # Switch to query mode.
        return self.stored_result(
            self.db.query(query_texts=[query], 
                          n_results=max_sources, 
                          where=where), 
            is_query = True)

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_exchanges_csv(self, query: str):
        return self.generate_answer(query, max_sources=100, where={"source": "exchanges.csv"})

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def generate_answer(self, query: str, max_sources: int = 10, 
                        where: Optional[dict] = None, passages: Optional[list[str]] = None):
        stored = self.get_documents_list(query, max_sources, where)
        query_oneline = query.replace("\n", " ")
        prompt = f"""You're an expert writer. You understand how to interpret html and markdown. You will accept the
        question below and answer based only on the passages. Never mention the passages in your answers. Be sure to 
        respond in concise sentences. Include all relevant background information when possible. If a passage is not 
        relevant to the answer you must ignore it. If no passage answers the question respond with: I don't know.

        QUESTION: {query_oneline}
        
        """
        # Add the retrieved documents to the prompt.
        stored_docs = [passage.docs for passage in stored]
        for passage in stored_docs if passages is None else stored_docs + passages:
            passage_oneline = passage.replace("\n", " ")
            prompt += f"PASSAGE: {passage_oneline}\n"
        # Generate the response.
        response = api.retriable(
            self.client.models.generate_content,
            model=api(Api.Model.GEN),
            config=self.config_temp,
            contents=prompt)
        # Check for generated code and store in memory.
        content = response.candidates[0].content
        if len(content.parts) > 1 and content.parts[0].executable_code:
            memory.append_code(prompt, content.parts)
        return response

    def stored_result(self, result, is_query: bool = False) -> list[VectorStoreResult]:
        try:
            results = []
            if len(result["documents"]) == 0:
                return results
            if isinstance(result["documents"][0], list):
                for i in range(len(result["documents"][0])):
                    obj = VectorStoreResult(docs=result["documents"][0][i],
                                            dist=result["distances"][0][i] if is_query else None,
                                            meta=result["metadatas"][0][i],
                                            store_id=result["ids"][0][i])
                    results.append(obj)
            else:
                results.append(
                    VectorStoreResult(docs=result["documents"][0],
                                      dist=result["distances"][0] if is_query else None,
                                      meta=result["metadatas"][0],
                                      store_id=result["ids"][0]))
            return results
        except Exception as e:
            raise e

## Wiki Grounding

In [14]:
# Define tool: wiki-grounding generation.
# - using gemini-2.0-flash for response generation
# - using a RAG-implementation to store groundings
# - create new groundings by similarity to topic
# - retrieve existing groundings by similarity to topic
class WikiGroundingGenerator:   
    def __init__(self, genai_client, rag_impl):
        self.client = genai_client
        self.rag = rag_impl
        with warnings.catch_warnings():
            warnings.simplefilter("ignore") # suppress beta-warning
            self.splitter = HTMLSemanticPreservingSplitter(
                headers_to_split_on=[("h2", "Main Topic"), ("h3", "Sub Topic")],
                separators=["\n\n", "\n", ". ", "! ", "? "],
                max_chunk_size=Api.Const.ChunkMax(),
                chunk_overlap=50,
                preserve_links=True,
                preserve_images=True,
                preserve_videos=True,
                preserve_audio=True,
                elements_to_preserve=["table", "ul", "ol", "code"],
                denylist_tags=["script", "style", "head"],
                custom_handlers={"code": self.code_handler},
            )

    def generate_answer(self, query: str, topic: str):
        stored = self.rag.get_wiki_documents(topic)
        if len(stored) > 0:
            return self.rag.generate_with_wiki_passages(query, topic, [chunk.docs for chunk in stored]).text
        else:
            pages = wikipedia.search(topic + " company")
            if len(pages) > 0:
                p_topic_match = 0.80
                for i in range(len(pages)):
                    if tqdm(api.similarity([topic + " company", pages[i]]) > p_topic_match, 
                            desc= "Score wiki search by similarity to topic"):
                        page_html = Api.get(f"https://en.wikipedia.org/wiki/{pages[i]}")
                        chunks = [chunk.page_content for chunk in self.splitter.split_text(page_html)]
                        self.rag.add_wiki_documents(topic, chunks)
                        return self.rag.generate_with_wiki_passages(query, topic, chunks).text
            return Api.Const.Stop()

    def code_handler(self, element: Tag) -> str:
        data_lang = element.get("data-lang")
        code_format = f"<code:{data_lang}>{element.get_text()}</code>"
        return code_format

## Search Grounding

In [15]:
# Define tool: search-grounding generation.
# - using gemini-2.0-flash with GoogleSearch tool for response generation
# - using a RAG-implementation to store groundings
# - create new groundings by exact match to topic
# - retrieve existing groundings by similarity to topic
class SearchGroundingGenerator:
    config_ground = types.GenerateContentConfig(
        tools=[types.Tool(google_search=types.GoogleSearch())],
        temperature=0.0
    )
    
    def __init__(self, genai_client, rag_impl):
        self.client = genai_client
        self.rag = rag_impl

    def generate_answer(self, query: str, topic: str):
        stored = self.rag.get_grounding_documents(query, topic)
        if len(stored) > 0:
            for i in range(len(stored)):
                meta_q = stored[i].meta["question"]
                p_ground_match = 0.95 # This can be really high ~ 95-97%
                if tqdm(api.similarity([query, meta_q]) > p_ground_match,
                        desc="Score similarity to stored grounding"):
                    return ast.literal_eval(stored[i].docs)
        return self.get_grounding(query, topic)

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_grounding(self, query: str, topic: str):
        contents = [types.Content(role="user", parts=[types.Part(text=query)])]
        contents += f"""
        You're a search assistant that provides answers to questions about {topic}.
        Do not discuss alternative topics of interest. Do not discuss similar topics.
        You will provide answers that discuss only {topic}. 
        You may discuss the owner or parent of {topic} when no other answer is possible.
        Otherwise respond with: I don't know."""
        response = api.retriable(self.client.models.generate_content, 
                                 model=api(Api.Model.GEN), 
                                 config=self.config_ground, 
                                 contents=contents)
        if response.candidates[0].grounding_metadata.grounding_supports is not None:
            if self.is_consistent(query, topic, response.text):
                self.rag.add_grounded_document(query, topic, response)
                return response.text 
        return Api.Const.Stop() # Empty grounding supports or not consistent in response

    def is_consistent(self, query: str, topic: str, model_response: str) -> bool:
        topic = topic.replace("'", "")
        id_strs = topic.split()
        if len(id_strs) == 1:
            matches = re.findall(rf"{id_strs[0]}[\s,.]+\S+", query)
            if len(matches) > 0:
                topic = matches[0]
        compound_match = re.findall(rf"{id_strs[0]}[\s,.]+\S+", model_response)
        model_response = model_response.replace("'", "")
        if len(compound_match) == 0 and topic in model_response:
            return True # not a compound topic id and exact topic match
        for match in compound_match:
            if topic not in match:
                return False
        return True # all prefix matches contained topic

## Rest Grounding

In [16]:
# Rest api-helpers to manage request-per-minute limits.
# - define an entry for each endpoint limit
# - init rest tool with limits to create blocking queues
# - apply a limit to requests with rest_tool.try_url
class ApiLimit(Enum):
    FINN = "finnhub.io",50
    POLY = "polygon.io",4 # (id_url,rpm)

class BlockingUrlQueue:
    on_cooldown = False
    cooldown = None
    cooldown_start = None
    
    def __init__(self, rest_fn: Callable, per_minute: int):
        self.per_minute_max = per_minute
        self.quota = per_minute
        self.rest_fn = rest_fn

    def push(self, rest_url: str):
        if not self.on_cooldown:
            self.cooldown = Timer(60, self.reset_quota)
            self.cooldown.start()
            self.cooldown_start = time.time()
            self.on_cooldown = True
        if self.quota > 0:
            self.quota -= 1
            time.sleep(0.034) # ~30 requests per second
            return self.rest_fn(rest_url)
        else:
            print(f"limited {self.per_minute_max}/min, waiting {self.limit_expiry()}s")
            time.sleep(max(self.limit_expiry(),0.5))
            return self.push(rest_url)

    def reset_quota(self):
        self.quota = self.per_minute_max
        self.on_cooldown = False
        self.cooldown_start = None

    def limit_expiry(self):
        if self.cooldown_start:
            return max(60-(time.time()-self.cooldown_start),0)
        return 0

In [17]:
# Define tool: rest-grounding generation.
# - using gemini-2.0-flash for response generation
# - using a RAG-implementation to store groundings
# - reduce long-context by chunked pre-processing
class RestGroundingGenerator:    
    limits = None

    def __init__(self, rag_impl, with_limits: bool):
        self.rag = rag_impl
        if with_limits:
            self.limits = {}
            for rest_api in ApiLimit:
                self.limits[rest_api.value[0]] = BlockingUrlQueue(Api.get, rest_api.value[1])

    def get_limit(self, rest_api: ApiLimit) -> Optional[BlockingUrlQueue]:
        return self.limits[rest_api.value[0]] if self.limits else None

    def basemodel(self, data: str, schema: BaseModel, from_lambda: bool = False) -> Optional[BaseModel]:
        try:
            if from_lambda:
                return schema(results=json.loads(data))
            return schema.model_validate_json(data)
        except Exception as e:
            raise e

    def dailycandle(self, data: str) -> Optional[DailyCandle]:
        try:
            candle = json.loads(data)
            if "from" not in candle:
                raise ValueError("not a dailycandle / missing value for date")
            agg = self.basemodel(data, Aggregate)
            return DailyCandle(from_date=candle["from"], 
                               status=agg.status.value, 
                               symbol=agg.symbol, 
                               open=agg.open, 
                               high=agg.high, 
                               low=agg.low, 
                               close=agg.close, 
                               volume=agg.volume, 
                               otc=agg.otc, 
                               preMarket=agg.preMarket, 
                               afterHours=agg.afterHours)
        except Exception as e:
            raise e

    @retry.Retry(timeout=600)
    def try_url(self, url: str, schema: BaseModel, as_lambda: bool, with_limit: Optional[BlockingUrlQueue],
                success_fn: Callable, *args, **kwargs):
        try:
            if self.limits is None:
                data = Api.get(url)
            elif with_limit:
                data = with_limit.push(url)
            if schema is DailyCandle:
                model = self.dailycandle(data)
            else:
                model = self.basemodel(data, schema, as_lambda)
        except Exception as e:
            try:
                print(f"try_url exception: {e}")
                if issubclass(schema, RestResultPoly):
                    return success_fn(*args, **kwargs, result=self.basemodel(data, RestResultPoly))
            except Exception as not_a_result:
                print(not_a_result)
            return StopGeneration()
        else:
            return success_fn(*args, **kwargs, model=model)

    def get_symbol_matches(self, with_content, by_name: bool, model: SymbolResult):
        matches = []
        max_failed_match = model.count if not by_name else 3
        p_desc_match = 0.92
        p_symb_match = 0.95
        if model.count > 0:
            for obj in tqdm(model.get(), desc="Score similarity to query"):
                if max_failed_match > 0:
                    desc = [with_content["q"].upper(), obj.description.split("-", -1)[0]]
                    symb = [with_content["q"].upper(), obj.symbol]
                    if by_name and api.similarity(desc) > p_desc_match: 
                        matches.append(obj.symbol)
                    elif not by_name and api.similarity(symb) > p_symb_match:
                        matches.append(obj.description)
                        max_failed_match = 0
                    else:
                        max_failed_match -= 1
        if len(matches) > 0:
            self.rag.add_api_document(with_content["query"], matches, with_content["q"], "get_symbol_1")
            return matches
        return Api.Const.Stop()

    def get_quote(self, with_content, model: Quote):
        quote = model.model_dump_json()
        self.rag.add_quote_document(with_content["query"], quote, with_content["symbol"], model.t, "get_quote_1")
        return quote

    def parse_financials(self, with_content, model: BasicFinancials):
        metric = list(model.metric.items())
        chunks = []
        # Chunk the metric data.
        for i in range(0, len(metric), Api.Const.MetricBatch()):
            batch = metric[i:i + Api.Const.MetricBatch()]
            chunks.append({"question": with_content["query"], "answer": batch})
        # Chunk the series data.
        for key in model.series.keys():
            series = list(model.series[key].items())
            for s in series:
                if api.token_count(s) <= Api.Const.ChunkMax():
                    chunks.append({"question": with_content["query"], "answer": s})
                else:
                    k = s[0]
                    v = s[1]
                    for i in range(0, len(v), Api.Const.SeriesBatch()):
                        batch = v[i:i + Api.Const.SeriesBatch()]
                        chunks.append({"question": with_content["query"], "answer": {k: batch}})
        self.rag.add_rest_chunks(chunks, topic=with_content["symbol"], source="get_financials_1")
        return chunks

    def parse_news(self, with_content, model: NewsResultFinn):
        if model.count > 0:
            metas = []
            for digest in model.get():
                pub_date = datetime.fromtimestamp(digest.datetime, tz=GeneratedEvent.tz()).strftime("%Y-%m-%d")
                metas.append({"publisher": digest.source,
                              "published_est": parse(pub_date).timestamp(),
                              "news_id": digest.id,
                              "related": digest.related})
            self.rag.add_rest_chunks(model.get(), topic=with_content["symbol"], source="get_news_1",
                                     ids=[f"{digest.id}+news" for digest in model.get()],
                                     meta_opt=metas, is_update=False)
            return [digest.summary().model_dump_json() for digest in model.get()]
        return Api.Const.Stop()

    def parse_news(self, with_content, model: Optional[NewsResultPoly] = None,
                   result: Optional[RestResultPoly] = None) -> tuple[list, str]: # list of summary, next list url
        if model and model.status in [RestStatus.OK, RestStatus.DELAY]:
            metas = []
            for news in model.get():
                pub_date = parse(news.published_utc).strftime("%Y-%m-%d")
                metas.append({"publisher": news.publisher.name,
                              "published_utc": parse(pub_date).timestamp(),
                              "news_id": news.id,
                              "related": json.dumps(news.tickers),
                              "keywords": json.dumps(news.keywords)})
            self.rag.add_rest_chunks(model.get(), topic=with_content["ticker"], source="get_news_2",
                                     ids=[news.id for news in model.get()],
                                     meta_opt=metas, is_update=False)
            return [news.summary().model_dump_json() for news in model.get()], model.next_url
        elif result:
            return result.model_dump_json()

    def parse_daily_candle(self, with_content, model: Optional[DailyCandle] = None,
                           result: Optional[RestResultPoly] = None):
        if model and model.status in [RestStatus.OK, RestStatus.DELAY]:
            self.rag.add_rest_chunks(
                chunks=[model],
                topic=with_content["stocksTicker"],
                source="daily_candle_2",
                meta_opt=[{"from_date": model.from_date, "adjusted": with_content["adjusted"]}])
            return model
        elif result:
            return result

    def parse_custom_candle(self, with_content, model: Optional[CustomCandle] = None,
                            result: Optional[RestResultPoly] = None):
        if model and model.status in [RestStatus.OK, RestStatus.DELAY]:
            metas = [{
                "timespan": with_content["timespan"],
                "adjusted": with_content["adjusted"],
                "from": with_content["from"],
                "to": with_content["to"]}]*model.count
            candles = [candle.model_dump_json() for candle in model.get()]
            self.rag.add_rest_chunks(
                chunks=candles,
                topic=with_content["stocksTicker"],
                source="custom_candle_2",
                meta_opt=metas)
            return candles
        elif result:
            return result.model_dump_json()

    def parse_overview(self, with_content, model: OverviewResult):
        overview = [model.get().model_dump_json()]
        self.rag.add_rest_chunks(chunks=overview, topic=with_content["ticker"], source="ticker_overview_2")
        return overview

    def parse_trends(self, with_content, model: TrendsResult):
        if model.count > 0:
            metas = [{"period": trend.period} for trend in model.get()]
            trends = [trend.model_dump_json() for trend in model.get()]
            self.rag.add_rest_chunks(trends, topic=with_content["symbol"], source="trends_1", meta_opt=metas)
            return trends
        return Api.Const.Stop()

    def augment_market_status(self, with_id: Optional[str], model: MarketStatusResult):
        if model.get().holiday != MarketSession.NA.value:
            self.rag.set_holiday_event(model.get().exchange)
        events = self.rag.generated_events(model.get().exchange)
        model.get().session = events.session()
        model.get().isOpen = events.is_open()
        meta = {"exchange": model.get().exchange,
                "last_close": events.last_close,
                "pre_open": events.pre_open,
                "reg_open": events.reg_open,
                "reg_close": events.reg_close,
                "post_close": events.post_close,
                "timestamp": events.timestamp }
        self.rag.add_rest_chunks([model.get()],
                                 topic="market_status",
                                 source="get_market_status_1",
                                 ids=[with_id] if with_id else None,
                                 meta_opt=[meta])
        return model.get().model_dump_json()

    def get_symbol(self, content, by_name: bool = True):
        return self.try_url(
            f"https://finnhub.io/api/v1/search?q={content['q']}&exchange={content['exchange']}&token={FINNHUB_API_KEY}",
            schema=SymbolResult,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.get_symbol_matches,
            with_content=content,
            by_name=by_name)

    def get_current_price(self, content):
        return self.try_url(
            f"https://finnhub.io/api/v1/quote?symbol={content['symbol']}&token={FINNHUB_API_KEY}",
            schema=Quote,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.get_quote,
            with_content=content)

    def get_market_status(self, content, store_id: Optional[str] = None):
        return self.try_url(
            f"https://finnhub.io/api/v1/stock/market-status?exchange={content['exchange']}&token={FINNHUB_API_KEY}",
            schema=MarketStatusResult,
            as_lambda=True,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.augment_market_status,
            with_id=store_id)

    def get_peers(self, content):
        return self.try_url(
            f"https://finnhub.io/api/v1/stock/peers?symbol={content['symbol']}&grouping={content['grouping']}&token={FINNHUB_API_KEY}",
            schema=PeersResult,
            as_lambda=True,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=lambda model: model)

    def get_basic_financials(self, content):
        return self.try_url(
            f"https://finnhub.io/api/v1/stock/metric?symbol={content['symbol']}&metric={content['metric']}&token={FINNHUB_API_KEY}",
            schema=BasicFinancials,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.parse_financials,
            with_content=content)

    def get_news_simple(self, content):
        return self.try_url(
            f"https://finnhub.io/api/v1/company-news?symbol={content['symbol']}&from={content['from']}&to={content['to']}&token={FINNHUB_API_KEY}",
            schema=NewsResultFinn,
            as_lambda=True,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.parse_news,
            with_content=content)

    def get_news_tagged(self, content):
        next_url = f"https://api.polygon.io/v2/reference/news?ticker={content['ticker']}&published_utc.gte={content['published_utc.gte']}&published_utc.lte={content['published_utc.lte']}&order={content['order']}&limit={content['limit']}&sort={content['sort']}&apiKey={POLYGON_API_KEY}"
        news = []
        while True:
            news_list, next_url = self.try_url(
                next_url,
                schema=NewsResultPoly,
                as_lambda=False,
                with_limit=self.get_limit(ApiLimit.POLY),
                success_fn=self.parse_news,
                with_content=content)
            news += news_list
            if next_url is None:
                break
            next_url += f"&apiKey={POLYGON_API_KEY}"
        return news

    def get_daily_candle(self, content):
        return self.try_url(
            f"https://api.polygon.io/v1/open-close/{content['stocksTicker']}/{content['date']}?adjusted={content['adjusted']}&apiKey={POLYGON_API_KEY}",
            schema=DailyCandle,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.POLY),
            success_fn=self.parse_daily_candle,
            with_content=content)

    def get_custom_candle(self, content):
        return self.try_url(
            f"https://api.polygon.io/v2/aggs/ticker/{content['stocksTicker']}/range/{content['multiplier']}/{content['timespan']}/{content['from']}/{content['to']}?adjusted={content['adjusted']}&sort={content['sort']}&limit={content['limit']}&apiKey={POLYGON_API_KEY}",
            schema=CustomCandle,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.POLY),
            success_fn=self.parse_custom_candle,
            with_content=content)

    def get_overview(self, content):
        return self.try_url(
            f"https://api.polygon.io/v3/reference/tickers/{content['ticker']}?apiKey={POLYGON_API_KEY}",
            schema=OverviewResult,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.POLY),
            success_fn=self.parse_overview,
            with_content=content)

    def get_trends_simple(self, content):
        return self.try_url(
            f"https://finnhub.io/api/v1/stock/recommendation?symbol={content['symbol']}&token={FINNHUB_API_KEY}",
            schema=TrendsResult,
            as_lambda=True,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.parse_trends,
            with_content=content)

## Callable Functions

In [18]:
# Callable functions in openapi schema.
decl_get_symbol_1 = types.FunctionDeclaration(
    name="get_symbol_1",
    description="""Search for the stock ticker symbol of a given company, security, isin or cusip. Each ticker
                   entry provides a description, symbol, and asset type. If this doesn't help you should try 
                   calling get_wiki_tool_response next.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": """A ticker symbol to search for."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["q", "exchange", "query"]
    }
)

decl_get_symbols_1 = types.FunctionDeclaration(
    name="get_symbols_1",
    description="""List all supported symbols and tickers. The results are filtered by exchange code.""",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter the results."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["exchange", "query"]
    }
)

decl_get_name_1 = types.FunctionDeclaration(
    name="get_name_1",
    description="""Search for the name associated with a stock ticker or symbol's company, security, isin or cusip. 
    Each ticker entry provides a description, matching symbol, and asset type.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": """The symbol or ticker to search for."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            },
            "company": {
                "type": "string",
                "description": "The company you're searching for."
            }
        },
        "required": ["q", "exchange", "query", "company"]
    }
)

decl_get_symbol_quote_1 = types.FunctionDeclaration(
    name="get_symbol_quote_1",
    description="""Search for the current price or quote of a stock ticker or symbol. The response is
                   provided in json format. Each response contains the following key-value pairs:
                   
                   c: Current price,
                   d: Change,
                  dp: Percent change,
                   h: High price of the day,
                   l: Low price of the day,
                   o: Open price of the day,
                  pc: Previous close price,
                   t: Epoch timestamp of price in seconds.

                   Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "The stock ticker symbol for a company, security, isin, or cusip." 
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            },
            "exchange": {
                "type": "string",
                "description": "The exchange code used to filter quotes. This must always be 'US'."
            }
        },
        "required": ["symbol", "query", "exchange"]
    }
)

decl_get_local_datetime = types.FunctionDeclaration(
    name="get_local_datetime",
    description="""Converts an array of timestamps from epoch time to the local timezone format. The result is an array
                   of date and time in locale appropriate format. Suitable for use in a locale appropriate response.
                   Treat this function as a vector function. Always prefer to batch timestamps for conversion. Use this
                   function to format date and time in your responses.""",
    parameters={
        "type": "object",
        "properties": {
            "t": {
                "type": "array",
                "description": """An array of timestamps in seconds since epoch to be converted. The order of
                                  timestamps matches the order of conversion.""",
                "items": {
                    "type": "integer"
                }
            }
        },
        "required": ["t"]
    }
)

decl_get_market_status_1 = types.FunctionDeclaration(
    name="get_market_status_1",
    description="""Get the current market status of global exchanges. Includes whether exchanges are open or closed.  
                   Also includes holiday details if applicable. The response is provided in json format. Each response 
                   contains the following key-value pairs:

                   exchange: Exchange code,
                   timezone: Timezone of the exchange,
                    holiday: Holiday event name, or null if it's not a holiday,
                     isOpen: Whether the market is open at the moment,
                          t: Epoch timestamp of status in seconds (Eastern Time),
                    session: The market session can be 1 of the following values: 
                    
                    pre-market,regular,post-market when open, or null if closed.
                    
                    Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            }
        },
        "required": ["exchange"]
    }
)

decl_get_market_session_1 = types.FunctionDeclaration(
    name="get_market_session_1",
    description="Get the current market session of global exchanges.",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            }
        },
        "required": ["exchange"]
    }
)

decl_get_company_peers_1 = types.FunctionDeclaration(
    name="get_company_peers_1",
    description="""Search for a company's peers. Returns a list of peers operating in the same country and in the same
                   sector, industry, or subIndustry. Each response contains the following key-value pairs: 
                   
                   symbol: The company's stock ticker symbol, 
                   peers: A list containing the peers.
                   
                   Each peers entry contains the following key-value pairs:
                   
                   symbol: The peer company's stock ticker symbol, 
                   name: The peer company's name.
                   
                   Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "The stock ticker symbol of a company to obtain peers."
            },
            "grouping": {
                "type": "string",
                "description": """This parameter may be one of the following values: sector, industry, subIndustry.
                                  Always use subIndustry unless told otherwise."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "grouping", "exchange", "query"]
    }
)

decl_get_exchange_codes_1 = types.FunctionDeclaration(
    name="get_exchange_codes_1",
    description="""Get a dictionary mapping all supported exchange codes to their names."""
)

decl_get_exchange_code_1 = types.FunctionDeclaration(
    name="get_exchange_code_1",
    description="""Search for the exchange code to use when filtering by exchange. The result will be one or
                   more exchange codes provided as a comma-separated string value.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": "Specifies which exchange code to search for."
            }
        },
        "required": ["q"]
    }
)

decl_get_financials_1 = types.FunctionDeclaration(
    name="get_financials_1",
    description="""Get company basic financials such as margin, P/E ratio, 52-week high/low, etc. Parse the response for 
                   key-value pairs in json format and interpret their meaning as stock market financial indicators.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "metric": {
                "type": "string",
                "description": "It must always be declared as the value 'all'"
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "metric", "query"]
    }
)

decl_get_daily_candlestick_2 = types.FunctionDeclaration(
    name="get_daily_candlestick_2",
    description="""Get a historical daily stock ticker candlestick / aggregate bar (OHLC). 
                   Includes historical daily open, high, low, and close prices. Also includes historical daily trade
                   volume and pre-market/after-hours trade prices. It provides the last trading days' data after 
                   11:59PM Eastern Time.""",
    parameters={
        "type": "object",
        "properties": {
            "stocksTicker": {
                "type": "string",
                "description": "The stock ticker symbol of a company to search for.",
            },
            "date": {
                "type": "string",
                "format": "date-time",
                "description": """The date of the requested candlestick in format YYYY-MM-DD."""
            },
            "adjusted": {
                "type": "string",
                "description": """May be true or false. Indicates if the results should be adjusted for splits.
                                  Use true unless told otherwise."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["stocksTicker", "date", "adjusted", "exchange", "query"]
    },
)

decl_get_company_news_1 = types.FunctionDeclaration(
    name="get_company_news_1",
    description="Retrieve the most recent news articles related to a specified ticker.",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company.",
            },
            "from": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD. It must be older than the parameter 'to'."""
            },
            "to": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD. It must be more recent than the parameter 'from'. The
                                  default value is today's date."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "from", "to", "query"]
    },
)

decl_get_custom_candlestick_2 = types.FunctionDeclaration(
    name="get_custom_candlestick_2",
    description="""Get a historical stock ticker candlestick / aggregate bar (OHLC) over a custom date range and 
                   time interval in Eastern Time. Includes historical open, high, low, and close prices. Also 
                   includes historical daily trade volume and pre-market/after-hours trade prices. It includes 
                   the last trading days' data after 11:59PM Eastern Time.""",
    parameters={
        "type": "object",
        "properties": {
            "stocksTicker": {
                "type": "string",
                "description": "The stock ticker symbol of a company to search for.",
            },
            "multiplier": {
                "type": "integer",
                "description": "This must be included and equal to 1 unless told otherwise."
            },
            "timespan": {
                "type": "string",
                "description": """The size of the candlestick's time window. This is allowed to be one of the following:
                                  second, minute, hour, day, week, month, quarter, or year. The default value is day."""
            },
            "from": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be older than the parameter 'to'."""
            },
            "to": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be more recent than the parameter 'from'. The 
                                  default is one weekday before get_last_market_close.
                                  Replace more recent dates with the default."""
            },
            "adjusted": {
                "type": "string",
                "description": """May be true or false. Indicates if the results should be adjusted for splits.
                                  Use true unless told otherwise."""
            },
            "sort": {
                "type": "string",
                "description": """This must be included. May be one of asc or desc. asc will sort by timestmap in 
                                  ascending order. desc will sort by timestamp in descending order."""
            },
            "limit": {
                "type": "integer",
                "description": """Set the number of base aggregates used to create this candlestick. This must be 5000 
                                  unless told to limit base aggregates to something else."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["stocksTicker", "multiplier", "timespan", "from", "to", "adjusted", "sort", "limit", "query"]
    },
)

decl_get_last_market_close = types.FunctionDeclaration(
    name="get_last_market_close",
    description="""Get the last market close of the specified exchange in Eastern Time. The response has already
                   been converted by get_local_datetime so this step should be skipped.""",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            }
        },
        "required": ["exchange"]
    }
)

decl_get_ticker_overview_2 = types.FunctionDeclaration(
    name="get_ticker_overview_2",
    description="""Retrieve comprehensive details for a single ticker symbol. It's a deep look into a company’s 
    fundamental attributes, including its primary exchange, standardized identifiers (CIK, composite FIGI, 
    share class FIGI), market capitalization, industry classification, and key dates. Also includes branding assets in
    the form of icons and logos.
    """,
    parameters={
        "type": "object",
        "properties": {
            "ticker": {
                "type": "string",
                "description": "Stock ticker symbol of a company."
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["ticker", "query"]
    }
)

decl_get_recommendation_trends_1 = types.FunctionDeclaration(
    name="get_recommendation_trends_1",
    description="""Get the latest analyst recommendation trends for a company.
                The data includes the latest recommendations as well as historical
                recommendation data for each month. The data is classified according
                to these categories: strongBuy, buy, hold, sell, and strongSell.
                The date of a recommendation indicated by the value of 'period'.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "query"]
    }
)

decl_get_news_with_sentiment_2 = types.FunctionDeclaration(
    name="get_news_with_sentiment_2",
    description="""Retrieve the most recent news articles related to a specified ticker. Each article includes 
                   comprehensive coverage. Including a summary, publisher information, article metadata, 
                   and sentiment analysis.""",
    parameters={
        "type": "object",
        "properties": {
            "ticker": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "published_utc.gte": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be older than the parameter 'published_utc.lte'. 
                                  The default value is one-month ago from today's date."""
            },
            "published_utc.lte": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be more recent than the parameter 'published_utc.gte'.
                                  The default is one weekday prior to get_last_market_close (excluding weekends).
                                  Replace more recent dates with the default."""
            },
            "order": {
                "type": "string",
                "description": """Must be desc for descending order, or asc for ascending order.
                                  When order is not specified the default is descending order.
                                  Ordering will be based on the parameter 'sort'."""
            },
            "limit": {
                "type": "integer",
                "description": """This must be included and equal to 1000 unless told otherwise."""
            },
            "sort": {
                "type": "string",
                "description": """The sort field used for ordering. This value must
                                  always be published_utc."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["limit", "ticker", "published_utc.gte", "published_utc.lte", "order", "sort", "query"]
    }
)

decl_get_rag_tool_response = types.FunctionDeclaration(
    name="get_rag_tool_response",
    description="""A database containing useful financial information. Always check here for answers first.""",
    parameters={
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "A question needing an answer. Asked as a simple string."
            }
        }
    }
)

decl_get_wiki_tool_response = types.FunctionDeclaration(
    name="get_wiki_tool_response",
    description="""Answers questions that still have unknown answers. Retrieve a wiki page related to a company, 
                   product, or service. Each web page includes detailed company information, financial indicators, 
                   tickers, symbols, history, and products and services.""",
    parameters={
        "type": "object",
        "properties": {
            "id": {
                "type": "string",
                "description": "The question's company or product. Just the name and no other details."
            },
            "q": {
                "type": "string",
                "description": "The complete, unaltered, query string."
            }
        },
        "required": ["id", "q"]
    }
)

decl_get_search_tool_response = types.FunctionDeclaration(
    name="get_search_tool_response",
    description="Answers questions that still have unknown answers. Use it after checking all your other tools.",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": "The question needing an answer. Asked as a simple string."
            },
            "id": {
                "type": "string",
                "description": "The question's company or product. In one word. Just the name and no other details."
            }
        },
        "required": ["q", "id"]
    }
)

In [19]:
# Import the finance api secret keys.

POLYGON_API_KEY = UserSecretsClient().get_secret("POLYGON_API_KEY")
FINNHUB_API_KEY = UserSecretsClient().get_secret("FINNHUB_API_KEY")

In [20]:
# Instantiate tools and load the exchange data from source csv.
# - Identifies exchanges by a 1-2 letter code which can be used to filter response data.
# - Also maps the exchange code to exchange details.
try:
    df = pandas.read_csv("/kaggle/input/exchanges/exchanges_src.csv")
except FileNotFoundError as e:
    df = pandas.read_csv("exchanges_src.csv") # local run
df = df.drop(["close_date"], axis=1).fillna("")
df.to_csv("exchanges.csv", index=False)
exchanges = CSVLoader(file_path="exchanges.csv", encoding="utf-8", csv_args={"delimiter": ","}).load()

# Prepare a RAG tool for use and add the exchange data.
tool_rag = RetrievalAugmentedGenerator(api.args.CLIENT, "finance")
tool_rag.add_documents_list(exchanges)

# Prepare a the grounding tools for use.
tool_wiki = WikiGroundingGenerator(api.args.CLIENT, tool_rag)
tool_ground = SearchGroundingGenerator(api.args.CLIENT, tool_rag)
tool_rest = RestGroundingGenerator(tool_rag, with_limits=True)

Generate document embedding: 0it [00:00, ?it/s]


## Function Calling Expert

In [21]:
# Implement the callable functions and function handler.

def ask_rag_tool(content):
    return tool_rag.generate_answer(content["question"]).text

def ask_wiki_tool(content):
    return tool_wiki.generate_answer(content["q"], content["id"])

def ask_search_tool(content):
    return tool_ground.generate_answer(content["q"], content["id"])

def get_exchange_codes_1(content):
    return tool_rag.get_exchange_codes()

def get_exchange_code_1(content):
    return tool_rag.get_exchange_codes(with_query=content)
    
def last_market_close(content):
    return tool_rag.last_market_close(content["exchange"])
    
def get_symbol_1(content, by_name: bool = True):
    stored = tool_rag.get_api_documents(content["query"], content["q"], "get_symbol_1")
    if len(stored) == 0:
        return tool_rest.get_symbol(content, by_name)
    return json.loads(stored[0].docs)

def get_symbols_1(content):
    return None # todo

def get_name_1(content):
    return get_symbol_1(content, by_name = False)

def get_quote_1(content):
    stored = tool_rag.get_api_documents(content["query"], content["symbol"], "get_quote_1")
    if tool_rag.generated_events(content["exchange"]).is_open():
        return get_current_price_1(content)
    elif len(stored) > 0:
        last_close = parse(tool_rag.last_market_close(content["exchange"])).timestamp()
        for quote in stored:
            if quote.meta["timestamp"] >= last_close:
                return [quote.docs for quote in stored]
    return get_current_price_1(content)

def get_current_price_1(content):
    return tool_rest.get_current_price(content)

def get_market_status_1(content):
    stored, has_update = tool_rag.get_market_status(content['exchange'])
    if has_update:
        with_id = stored[0].store_id if len(stored) > 0 else None
        return tool_rest.get_market_status(content, with_id)
    return stored[0].docs

def get_session_1(content):
    return json.loads(get_market_status_1(content))["session"]

def get_peers_1(content):
    stored = tool_rag.get_peers_document(content["query"], content["symbol"], content['grouping'])
    if len(stored) == 0:
        peers = tool_rest.get_peers(content)
        if peers.count > 0:
            names = []
            for peer in peers.get():
                if peer == content["symbol"]:
                    continue # skip including the query symbol in peers
                name = get_name_1(dict(q=peer, exchange=content["exchange"], query=content["query"]))
                if name != Api.Const.Stop():
                    data = {"symbol": peer, "name": name}
                    names.append(data)
            tool_rag.add_peers_document(content["query"], names, content["symbol"], "get_peers_1", content['grouping'])
            return names
        return Api.Const.Stop()
    return json.loads(stored[0].docs)["peers"]

def local_datetime(content):
    local_t = []
    for timestamp in content["t"]:
        local_t.append(local_date_from_epoch(timestamp))
    return local_t

def local_date_from_epoch(timestamp):
    if len(str(timestamp)) == 13:
        return datetime.fromtimestamp(timestamp/1000, tz=GeneratedEvent.tz()).strftime('%c')
    else:
        return datetime.fromtimestamp(timestamp, tz=GeneratedEvent.tz()).strftime('%c')

def get_financials_1(content):
    stored = tool_rag.get_basic_financials(content["query"], content["symbol"], "get_financials_1")
    if len(stored) == 0:
        return tool_rest.get_basic_financials(content)
    return [chunk.docs for chunk in stored]

def get_news_1(content):
    stored = tool_rag.get_api_documents(content["query"], content["symbol"], "get_news_1")
    if len(stored) == 0:
        return tool_rest.get_news_simple(content)
    return [NewsTypeFinn.model_validate_json(news.docs).summary().model_dump_json() for news in stored]

def get_daily_candle_2(content):
    stored = tool_rag.get_api_documents(
        query=content["query"], topic=content["stocksTicker"], source="daily_candle_2", 
        meta_opt=[{"from_date": content["date"], "adjusted": content["adjusted"]}])
    if len(stored) == 0:
        candle = tool_rest.get_daily_candle(content)
        # Attempt to recover from choosing a holiday.
        candle_date = parse(content["date"])
        if candle.status is RestStatus.NONE and candle_date.weekday() == 0 or candle_date.weekday() == 4:
            if candle_date.weekday() == 0: # index 0 is monday, index 4 is friday
                content["date"] = candle_date.replace(day=candle_date.day-3).strftime("%Y-%m-%d")
            else:
                content["date"] = candle_date.replace(day=candle_date.day-1).strftime("%Y-%m-%d")
            return get_daily_candle_2(content)
        return candle.model_dump_json()
    return [json.loads(candle.docs) for candle in stored]

def get_custom_candle_2(content):
    stored = tool_rag.get_api_documents(
        query=content["query"], topic=content["stocksTicker"], source="custom_candle_2", 
        meta_opt=[{
            "timespan": content["timespan"],
            "adjusted": content["adjusted"],
            "from": content["from"],
            "to": content["to"]}])
    if len(stored) == 0:
        return tool_rest.get_custom_candle(content)
    return [json.loads(candle.docs) for candle in stored]

def get_overview_2(content):
    stored = tool_rag.get_api_documents(content["query"], content["ticker"], "ticker_overview_2")
    if len(stored) == 0:
        return tool_rest.get_overview(content)
    return json.loads(stored[0].docs)

def get_trends_1(content):
    stored = tool_rag.get_api_documents(content["query"], content["symbol"], "trends_1")
    if len(stored) == 0:
        return tool_rest.get_trends_simple(content)
    return [json.loads(trend.docs) for trend in stored]

def get_news_2(content):
    timestamp_from = parse(content["published_utc.gte"]).timestamp()
    timestamp_to = parse(content["published_utc.lte"]).timestamp()
    news_from = tool_rag.get_api_documents(
        content["query"], content["ticker"], "get_news_2", [{"published_utc": timestamp_from}])
    news_to = tool_rag.get_api_documents(
        content["query"], content["ticker"], "get_news_2", [{"published_utc": timestamp_to}])
    if len(news_from) > 0 and len(news_to) > 0:
        stored = tool_rag.get_api_documents(
            content["query"], content["ticker"], "get_news_2",
            [{"published_utc": {"$gte": timestamp_from}},
             {"published_utc": {"$lte": timestamp_to}}])
        return [NewsTypePoly.model_validate_json(news.docs).summary().model_dump_json() for news in stored]
    return tool_rest.get_news_tagged(content)
        
finance_tool = types.Tool(
    function_declarations=[
        decl_get_symbol_1,
        decl_get_symbols_1,
        decl_get_name_1,
        decl_get_symbol_quote_1,
        decl_get_market_status_1,
        decl_get_market_session_1,
        decl_get_company_peers_1,
        decl_get_local_datetime,
        decl_get_last_market_close,
        decl_get_exchange_codes_1,
        decl_get_exchange_code_1,
        decl_get_financials_1,
        decl_get_daily_candlestick_2,
        decl_get_custom_candlestick_2,
        decl_get_ticker_overview_2,
        decl_get_recommendation_trends_1,
        decl_get_news_with_sentiment_2,
        decl_get_rag_tool_response,
        decl_get_wiki_tool_response,
        decl_get_search_tool_response
    ]
)

function_handler = {
    "get_symbol_1": get_symbol_1,
    "get_symbols_1": get_symbols_1,
    "get_name_1": get_name_1,
    "get_symbol_quote_1": get_quote_1,
    "get_market_status_1": get_market_status_1,
    "get_market_session_1": get_session_1,
    "get_company_peers_1": get_peers_1,
    "get_local_datetime": local_datetime,
    "get_last_market_close": last_market_close,
    "get_exchange_codes_1": get_exchange_codes_1,
    "get_exchange_code_1": get_exchange_code_1,
    "get_financials_1": get_financials_1,
    "get_daily_candlestick_2": get_daily_candle_2,
    "get_custom_candlestick_2": get_custom_candle_2,
    "get_ticker_overview_2": get_overview_2,
    "get_recommendation_trends_1": get_trends_1,
    "get_news_with_sentiment_2": get_news_2,
    "get_rag_tool_response": ask_rag_tool,
    "get_wiki_tool_response": ask_wiki_tool,
    "get_search_tool_response": ask_search_tool
}

In [22]:
# Implement the function calling expert.
# Define the system prompt.
instruction = f"""You are a helpful and informative bot that answers finance and stock market questions. 
Only answer the question asked and do not change topic. While the answer is still
unknown you must follow these rules for predicting function call order:

RULE#1: Always consult your other functions before get_search_tool_response.
RULE#2: Always consult get_wiki_tool_response before get_search_tool_response.
RULE#3: Always consult get_search_tool_response last.
RULE#4: Always convert timestamps with get_local_datetime and use the converted date/time in your response.
RULE#5: Always incorporate as much useful information from tools and functions in your response."""

def get_response():
    # Enable system prompt, function calling and minimum-randomness.
    config_fncall = types.GenerateContentConfig(
        system_instruction=instruction,
        tools=[finance_tool],
        temperature=0.0
    )
    memory.response = api.retriable(
        api.args.CLIENT.models.generate_content,
        model=api(Api.Model.GEN),
        config=config_fncall,
        contents=memory.contents)

def retry_last_send():
    api.generation_fail()
    time.sleep(api.dt_between)
    get_response()

@retry.Retry(
    predicate=is_retriable,
    initial=2.0,
    maximum=64.0,
    multiplier=2.0,
    timeout=600,
)
def send_message(prompt):
    #display(Markdown("#### Prompt"))
    #print(prompt, "\n")
    memory.set_prompt(prompt)
    # Handle cases with multiple chained function calls.
    function_calling_in_process = True
    # Send the initial user prompt and function declarations.
    get_response()
    while function_calling_in_process:
        try:
            response_parts = memory.response.candidates[0].content.parts
            # A summary response never includes function calls.
            if not any(part.function_call for part in response_parts):
                memory.set_summary("\n".join(e.text for e in response_parts))
                function_calling_in_process = False
                break # The function calling chain is complete.
            else:
                # A part can be a function call or reasoning-step.
                for part in response_parts:
                    if function_call := part.function_call:
                        # Extract the function call.
                        fn_name = function_call.name
                        #display(Markdown("#### Predicted function name"))
                        #print(fn_name, "\n")
                        # Extract the function call arguments.
                        fn_args = {key: value for key, value in function_call.args.items()}
                        #display(Markdown("#### Predicted function arguments"))
                        #print(fn_args, "\n")
                        # Call the predicted function.
                        print("send_message: get function response")
                        api_response = function_handler[fn_name](fn_args)[:20000] # Stay within the input token limit
                        #display(Markdown("#### API response"))
                        #print(api_response[:500], "...", "\n")
                        # Create an API response part.
                        api_response_part = types.Part.from_function_response(
                            name=fn_name,
                            response={"content": api_response},
                        )
                        memory.update_contents(function_call, api_response_part)
                    else:
                        #display(Markdown("#### Natural language reasoning step"))
                        #print(part.text)
                        memory.set_reason(part.text)
                print("send_message: updating state")
                get_response() # Send the updated prompt.
                print("send_message: got a response")
        except Exception as e:
            if isinstance(response_parts, list):
                print("send_message: generated wrong function arguments")
            retry_last_send()
            
    # Show the final natural language summary.
    display(Markdown("#### Natural language response"))
    display(Markdown(memory.summary))

# RAG Baseline Check

In [23]:
response = tool_rag.get_exchanges_csv(
    """Give me a dictionary in string form. It must contain key:value pairs mapping 
    exchange code to name. Just the dictionary string in pretty form.""")
print(response.candidates[0].content.parts[-1].text)

response = tool_rag.get_exchanges_csv(
    """What is the Germany exchange code? Return only the exchange codes as a simple 
    comma separated value that I can copy.""")
print(response.candidates[0].content.parts[-1].text, "\n")

response = tool_rag.get_exchanges_csv("What are the Germany exchanges and thier corresponding exchange codes?")
print(response.text, "\n")

response = tool_rag.generate_answer("What are Google's stock ticker symbols?")
print(response.text, "\n")

response = tool_rag.generate_answer("What is Facebook's stock ticker symbol?")
print(response.text, "\n")

response = tool_rag.get_exchanges_csv("What are the US exchange operating hours?")
print(response.text, "\n")

response = tool_rag.get_exchanges_csv(
    f"""Answer based on your knowledge of exchange operating hours.
    Do not answer in full sentences. Omit all chat and provide the answer only.
    The fields pre_market and post_market both represent extended operating hours.

    The current date and time: {datetime.now(GeneratedEvent.tz()).strftime('%c')}

    Weekdays are: Mon, Tue, Wed, Thu, Fri.
    On weekdays all exchanges open after pre-market and regular hours.
    On weekdays all exchanges close after regular and post-market hours.
    
    Weekends are: Sat, Sun.
    Always exclude weekends from exchange operating hours.
    A list of holidays in date format mm-dd-yyyy: {tool_rag.holidays["US"]}
    Always exclude holidays from exchange operating hours.
    When the answer is a holiday use the prior weekday for close.
    When the answer is a holiday use the next weekday for open.
    
    Consider the US exchange's operating hours.
    Provide the most recent weekday's close including post_market hours.
    
    Answer with a date that uses this format: '%a %b %d %X %Y'.""")
print(response.candidates[0].content.parts[-1].text)

```
{
    "VN": "Vietnam exchanges including HOSE, HNX and UPCOM",
    "AD": "ABU DHABI SECURITIES EXCHANGE",
    "US": "US exchanges (NYSE, Nasdaq)",
    "CO": "OMX NORDIC EXCHANGE COPENHAGEN A/S",
    "QA": "QATAR EXCHANGE",
    "BA": "BOLSA DE COMERCIO DE BUENOS AIRES",
    "MX": "BOLSA MEXICANA DE VALORES (MEXICAN STOCK EXCHANGE)",
    "PR": "PRAGUE STOCK EXCHANGE",
    "HK": "HONG KONG EXCHANGES AND CLEARING LTD",
    "CA": "Egyptian Stock Exchange",
    "AX": "ASX - ALL MARKETS",
    "SX": "DEUTSCHE BOERSE Stoxx",
    "KQ": "KOREA EXCHANGE (KOSDAQ)",
    "DB": "DUBAI FINANCIAL MARKET",
    "PM": "Philippine Stock Exchange",
    "KS": "KOREA EXCHANGE (STOCK MARKET)",
    "ST": "NASDAQ OMX NORDIC STOCKHOLM",
    "DU": "BOERSE DUESSELDORF",
    "TL": "NASDAQ OMX TALLINN",
    "AT": "ATHENS EXCHANGE S.A. CASH MARKET",
    "SW": "SWISS EXCHANGE",
    "LS": "NYSE EURONEXT - EURONEXT LISBON",
    "SI": "SINGAPORE EXCHANGE",
    "RG": "NASDAQ OMX RIGA",
    "CR": "CARACAS STOCK EXCHANGE"

# SC1 Baseline Check

In [24]:
# Wait 59s for rate-limits to reset on FREE-tier.
if api.args.API_LIMIT is Api.Limit.FREE.value:
    print("Gemini API limit is FREE. Waiting 59s...")
    time.sleep(59)

Gemini API limit is FREE. Waiting 59s...
Api.refill_rpm 10


In [25]:
send_message("What is the current session for US exchanges?")

send_message: get function response


Generate US->MarketEvent.PRE_OPEN:   0%|          | 0/1 [00:00<?, ?it/s]

Api.refill_rpm 10
caught exception of type <class 'google.api_core.exceptions.RetryError'>
Timeout of 120.0s exceeded, last exception: 503 UNAVAILABLE. {'error': {'code': 503, 'message': 'The service is currently unavailable.', 'status': 'UNAVAILABLE'}}
send_message: generated wrong function arguments
Api.generation_fail.next_model: model is now gemini-2.5-flash-preview-09-2025
send_message: get function response



Generate US->MarketEvent.LAST_CLOSE: 100%|██████████| 1/1 [00:03<00:00,  3.10s/it]

Generate US->MarketEvent.PRE_OPEN: 100%|██████████| 1/1 [00:03<00:00,  3.67s/it]

Generate US->MarketEvent.REG_OPEN: 100%|██████████| 1/1 [00:03<00:00,  3.06s/it]

Generate US->MarketEvent.REG_CLOSE: 100%|██████████| 1/1 [00:03<00:00,  3.67s/it]

Generate US->MarketEvent.POST_CLOSE: 100%|██████████| 1/1 [00:03<00:00,  3.16s/it]

Upsert chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response


#### Natural language response

The current market session for US exchanges is pre-market.

In [26]:
send_message("What is the US market status?")

Api.generation_fail.next_model: model is now gemini-2.0-flash-exp
send_message: get function response
send_message: updating state
send_message: got a response


#### Natural language response

The US market is currently open for pre-market trading. The timestamp for this status is Monday, December 1, 2025 at 04:50:17 AM Eastern Time. There is no holiday today.


In [27]:
send_message("When was the last US market close?")

send_message: get function response
send_message: updating state
send_message: got a response


#### Natural language response

The last US market close was on Fri Nov 28 20:00:00 2025.


In [28]:
send_message("What is Apple's stock ticker?")

send_message: get function response



Generate US->MarketEvent.PRE_OPEN:   0%|          | 0/1 [10:28<?, ?it/s]

Score similarity to query: 100%|██████████| 10/10 [00:00<00:00, 12.68it/s][A
Generate api embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response


#### Natural language response

Apple's stock ticker is AAPL.


In [29]:
send_message("What is the current price of Amazon stock? Display the result as a json string in markdown.")

send_message: get function response


Generate quote embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response
send_message: get function response
send_message: updating state
Api.generation_fail.next_model: model is now gemini-2.0-flash
send_message: got a response


#### Natural language response

The current price of Amazon stock (AMZN) is $233.22 as of Fri Nov 28 2025 16:00:00. The change from the previous close is $4.06, representing a 1.7717% increase. The high price of the day is $233.285, and the low price is $230.22. The opening price for the day was $231.24, and the previous close price was $229.16.

```json
{
"c": 233.22,
"d": 4.06,
"dp": 1.7717,
"h": 233.285,
"l": 230.22,
"o": 231.24,
"pc": 229.16,
"t": 1764363600
}
```

In [30]:
send_message("""Show me Apple's basic financials and help me understand key performance metrics. 
How has the stock performed?""")

send_message: get function response
Api.zero_error: model is now gemini-2.5-flash


Upsert chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response


#### Natural language response

Apple's basic financials reveal several key performance metrics:

**Profitability:**
*   **Gross Margin:** Apple's gross margin is currently 46.91% (TTM - Trailing Twelve Months), indicating a healthy profit after the cost of goods sold. This has been consistently strong, with a 5-year average of 44.47%.
*   **Net Profit Margin:** The net profit margin is 26.92% (TTM), showing that Apple retains a significant portion of its revenue as profit. The 5-year average is 25.48%.
*   **Operating Margin:** The operating margin stands at 31.97% (TTM), reflecting efficient management of operating expenses. The 5-year average is 30.67%.

**Efficiency:**
*   **Asset Turnover:** The asset turnover ratio is 1.2186 (TTM), suggesting that Apple generates $1.2186 in revenue for every dollar of assets.
*   **Inventory Turnover:** Apple's inventory turnover is 33.9834 (TTM), indicating that the company sells and replaces its inventory approximately 34 times a year. This is a very high turnover, which is typical for a company with a strong supply chain and high-demand products.
*   **Receivables Turnover:** The receivables turnover is 11.3725 (TTM), meaning Apple collects its receivables about 11 times a year.

**Liquidity:**
*   **Current Ratio:** The current ratio is 0.8933, which is below 1.0. This suggests that Apple's current assets are not sufficient to cover its current liabilities. However, for a company like Apple with strong cash flow and quick inventory turnover, a current ratio below 1.0 might not be a major concern.
*   **Quick Ratio:** The quick ratio is 0.8588, also below 1.0, reinforcing the observation from the current ratio.

**Solvency:**
*   **Long Term Debt/Equity:** This ratio is 1.0623, indicating that Apple has slightly more long-term debt than equity.
*   **Total Debt/Total Equity:** This ratio is 1.338, suggesting that Apple uses a significant amount of debt to finance its assets relative to its equity.

**Stock Performance:**
*   **52-Week High:** $280.38 (reached on 2025-11-25)
*   **52-Week Low:** $169.2101 (reached on 2025-04-08)
*   **52-Week Price Return Daily:** 18.6293%
*   **Year-to-Date Price Return Daily:** 11.3529%
*   **13-Week Price Return Daily:** 19.9045%
*   **26-Week Price Return Daily:** 39.2788%
*   **5-Day Price Return Daily:** 2.711%
*   **Month-to-Date Price Return Daily:** 3.1364%

**Valuation:**
*   **P/E Ratio (TTM):** 36.7859, which is relatively high, suggesting investors are willing to pay a premium for Apple's earnings.
*   **Forward P/E:** 33.862031136553547
*   **P/B Ratio:** 55.8825, indicating the stock is trading at a very high multiple of its book value.
*   **P/S Ratio (TTM):** 9.9009
*   **PEG Ratio (TTM):** 1.6255390214864598

**Earnings and Dividends:**
*   **EPS (TTM):** $7.4593000000000007
*   **Dividend Per Share Annual:** $1.0318
*   **Current Dividend Yield TTM:** 0.3743%
*   **Payout Ratio (TTM):** 13.77%, indicating that Apple pays out a small portion of its earnings as dividends.

**Overall Performance:**
Apple's stock has performed well over various periods, with positive returns across 5-day, month-to-date, 13-week, 26-week, 52-week, and year-to-date measures. The stock is trading significantly above its 52-week low and close to its 52-week high. The high P/E and P/B ratios suggest that the market has high expectations for Apple's future growth and profitability.

In [31]:
send_message("I need Apple's daily candlestick from 2025-05-05")

send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response


Upsert chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response


#### Natural language response

Apple's daily candlestick data for 2025-05-05 shows an opening price of 203.1, a high of 204.1, a low of 198.21, and a closing price of 198.89. The trading volume for the day was 69,018,452. The pre-market price was 205.0 and the after-hours price was 198.6.

In [32]:
send_message("Tell me who are Apple's peers?")

send_message: get function response


Score similarity to query: 100%|██████████| 5/5 [00:00<00:00, 26.70it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  5.78it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  5.46it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  5.57it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 5/5 [00:00<00:00, 25.49it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  5.41it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 2/2 [00:00<00:00, 10.31it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  5.64it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 2/2 [00:00<00:00, 11.02it/s]

send_message: updating state
send_message: got a response


#### Natural language response

Apple's peers include DELL TECHNOLOGIES -C (DELL), WESTERN DIGITAL CORP (WDC), SANDISK CORP (SNDK), PURE STORAGE INC - CLASS A (PSTG), HEWLETT PACKARD ENTERPRISE (HPE), HP INC (HPQ), NETAPP INC (NTAP), SUPER MICRO COMPUTER INC (SMCI), IONQ INC (IONQ), QUANTUM COMPUTING INC (QUBT), and COMPOSECURE INC-A (CMPO).

In [33]:
send_message("Tell me who are Amazon's peers?")

send_message: get function response


Score similarity to query: 100%|██████████| 2/2 [00:00<00:00,  5.47it/s]
Generate api embedding: 0it [00:00, ?it/s]


send_message: updating state
Api.generation_fail.next_model: model is now gemini-2.5-flash-preview-09-2025
send_message: got a response
send_message: get function response


Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  5.54it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 2/2 [00:00<00:00, 10.22it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  5.30it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 5/5 [00:00<00:00, 26.90it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 11/11 [00:00<00:00, 59.33it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 2/2 [00:00<00:00, 11.12it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  5.62it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  5.53it/s]


Api.zero_error: model is now gemini-2.5-flash
caught exception of type <class 'google.api_core.exceptions.RetryError'>
Timeout of 120.0s exceeded, last exception: 503 UNAVAILABLE. {'error': {'code': 503, 'message': 'The service is currently unavailable.', 'status': 'UNAVAILABLE'}}
send_message: generated wrong function arguments
Api.generation_fail.next_model: model is now gemini-2.5-flash-preview-09-2025
send_message: get function response


Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  5.50it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  5.22it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  5.44it/s]
Generate api embedding: 0it [00:00, ?it/s]
Generate peers embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response


#### Natural language response

Amazon's peers in the **Internet & Direct Marketing Retail** sub-industry are:

*   **Coupang Inc** (CPNG)
*   **eBay Inc** (EBAY)
*   **Dillard's Inc-CL A** (DDS)
*   **Ollie's Bargain Outlet Holdings** (OLLI)
*   **Macy's Inc** (M)
*   **Etsy Inc** (ETSY)
*   **Kohl's Corp** (KSS)
*   **Pattern Group Inc-CL A** (PTRN)
*   **Savers Value Village Inc** (SVV)
*   **Groupon Inc** (GRPN)

In [34]:
send_message("""Locate Apple's stock ticker, then download recommendation trends of all Apple's peers by sub-industry, 
and then finally compare them.""")

send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response


Upsert chunks embedding: 0it [00:00, ?it/s]


send_message: get function response


Upsert chunks embedding: 0it [00:00, ?it/s]


send_message: get function response


...

send_message: updating state
send_message: got a response
send_message: get function response


Upsert chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response
send_message: get function response
send_message: updating state
Api.generation_fail.next_model: model is now gemini-2.0-flash-exp
send_message: got a response


#### Natural language response

Okay, I have downloaded the recommendation trends for Apple (AAPL) and its peers in the sub-industry: DELL, WDC, SNDK, PSTG, HPE, HPQ, NTAP, SMCI, IONQ, QUBT, and CMPO.

Here's a summary of their recommendation trends for the last four months (December 2025, November 2025, October 2025, and September 2025):

*   **AAPL:** Strong Buy recommendations consistently around 15, Buy recommendations around 22-23, Hold recommendations around 15-17, and Sell recommendations around 2-3.
*   **DELL:** Strong Buy recommendations consistently around 7-8, Buy recommendations around 16-18, Hold recommendations around 6-7, and Sell recommendations around 0-1.
*   **WDC:** Strong Buy recommendations consistently around 6, Buy recommendations consistently around 19, Hold recommendations around 6-7, and Sell recommendations consistently at 0.
*   **SNDK:** Strong Buy recommendations consistently around 6-7, Buy recommendations around 10-11, Hold recommendations around 6-8, and Sell recommendations consistently at 0.
*   **PSTG:** Strong Buy recommendations consistently around 6-7, Buy recommendations around 12-13, Hold recommendations around 6-8, and Sell recommendations consistently at 1.
*   **HPE:** Strong Buy recommendations consistently around 6, Buy recommendations around 7-8, Hold recommendations around 10-13, and Sell recommendations consistently at 0.
*   **HPQ:** Strong Buy recommendations consistently around 1-2, Buy recommendations around 1-4, Hold recommendations around 14-16, and Sell recommendations around 1-6.
*   **NTAP:** Strong Buy recommendations consistently around 3, Buy recommendations around 9-10, Hold recommendations consistently around 14-15, and Sell recommendations consistently at 0.
*   **SMCI:** Strong Buy recommendations consistently around 2-4, Buy recommendations consistently around 10, Hold recommendations around 9-11, and Sell recommendations consistently at 3.
*   **IONQ:** Strong Buy recommendations consistently around 2, Buy recommendations around 9-10, Hold recommendations around 3-4, and Sell recommendations consistently at 0.
*   **QUBT:** Strong Buy recommendations consistently around 2, Buy recommendations around 4-5, Hold recommendations consistently around 2, and Sell recommendations consistently at 0.
*   **CMPO:** Strong Buy recommendations consistently around 2, Buy recommendations consistently around 8, Hold recommendations around 1-2, and Sell recommendations around 0-1.

**Comparison:**

Overall, Apple (AAPL) has a stronger buy recommendation trend compared to its peers, with a higher number of both Strong Buy and Buy recommendations. HPQ has the weakest buy recommendation trend.


In [35]:
send_message("""Tell me Amazon's current share price and provide candlestick data for the past month. 
Sort the data in descending order by date. Format the prices consistently as currency. 
Round prices to two decimal places. 
Present the data with multiple columns for display in markdown. 
Discuss and provide details about any patterns you notice in the price data. 
Correlate recent patterns with news over the same date range.""")

send_message: get function response


Generate quote embedding: 0it [00:00, ?it/s]


send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response


Upsert chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response
send_message: get function response


Add chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
Api.zero_error: model is now gemini-2.5-flash
send_message: got a response


#### Natural language response

As of November 28, 2025, at 16:00 EST, Amazon's current share price is $233.22.

Here's the candlestick data for Amazon (AMZN) over the past month (November 1, 2025, to November 28, 2025), sorted in descending order by date:

| Date       | Open    | High    | Low     | Close   | Volume      |
| ---------- | ------- | ------- | ------- | ------- | ----------- |
| 2025-11-28 | $231.24 | $233.29 | $230.22 | $233.22 | 20,250,425  |
| 2025-11-27 | $230.74 | $231.75 | $228.77 | $229.16 | 38,497,719  |
| 2025-11-26 | $226.38 | $230.52 | $223.80 | $229.67 | 39,379,339  |
| 2025-11-25 | $222.56 | $227.33 | $222.27 | $226.28 | 54,318,223  |
| 2025-11-24 | $216.35 | $222.21 | $215.18 | $220.69 | 68,490,453  |
| 2025-11-21 | $227.05 | $227.41 | $216.74 | $217.14 | 50,308,862  |
| 2025-11-20 | $223.74 | $223.74 | $218.52 | $222.69 | 58,335,353  |
| 2025-11-19 | $228.10 | $230.20 | $222.42 | $222.55 | 60,608,442  |
| 2025-11-18 | $233.25 | $234.60 | $229.19 | $232.87 | 59,918,908  |
| 2025-11-17 | $235.06 | $238.73 | $232.89 | $234.69 | 38,956,619  |
| 2025-11-14 | $243.05 | $243.75 | $236.50 | $237.58 | 41,401,638  |
| 2025-11-13 | $250.24 | $250.37 | $243.75 | $244.20 | 31,190,063  |
| 2025-11-12 | $248.41 | $249.75 | $247.23 | $249.10 | 23,563,960  |
| 2025-11-11 | $248.34 | $251.75 | $245.59 | $248.40 | 36,476,474  |
| 2025-11-07 | $242.90 | $244.90 | $238.49 | $244.41 | 46,374,294  |
| 2025-11-06 | $249.16 | $250.38 | $242.17 | $243.04 | 46,004,201  |
| 2025-11-05 | $249.03 | $251.00 | $246.16 | $250.20 | 40,610,602  |
| 2025-11-04 | $250.38 | $257.01 | $248.66 | $249.32 | 51,546,311  |
| 2025-11-03 | $255.36 | $258.60 | $252.90 | $254.00 | 95,997,714  |

### Analysis of Price Data and Correlation with Recent News:

**Overall Trend:**
The stock price generally declined over the past month, starting at a high of around $255 in early November and decreasing to around $233 by the end of November.

**Patterns:**

*   **Early November Peak:** The stock started the month near its highest point before experiencing a decline.
*   **Mid-Month Dip:** There was a noticeable dip in the middle of November, with the price fluctuating before showing a slight recovery towards the end of the month.
*   **Late November Recovery:** The stock showed some signs of recovery in the last few days of November, although it did not reach the levels seen at the beginning of the month.

**News Correlation:**

*   **Early November Optimism:** News from early November reflected some optimism, with analysts highlighting Amazon as a top AI stock and potential for long-term growth. The stock was also mentioned as a key holding in several investment portfolios.
*   **Mid-November Concerns:** As the month progressed, news articles began to express concerns about AI investments, potential market corrections, and workforce reductions at Amazon. There were also reports about increased competition in the cloud computing space.
*   **Late November Mixed Sentiment:** Towards the end of November, the news presented a mixed sentiment. Some articles highlighted Amazon's strong position in e-commerce and cloud services, while others pointed out challenges such as increased competition and potential overspending on AI infrastructure.

**Specific News Events:**

*   **AI and Cloud Computing:** Many news articles focused on Amazon's AI investments and its cloud computing business (AWS). Positive reports about AWS growth and strategic AI partnerships (e.g., with OpenAI) seemed to support the stock, while concerns about overspending on AI infrastructure and increased competition in the cloud space may have contributed to downward pressure.
*   **Market Corrections and Economic Uncertainty:** Broader market concerns, such as potential market corrections and economic uncertainty, also appeared to influence Amazon's stock price. Reports about workforce reductions and cautious investment strategies may have reflected these concerns.
*   **Analyst Ratings and Investment Firm Actions:** Analyst ratings and investment firm actions, such as Warren Buffett's investment in Alphabet and various firms adjusting their positions in Amazon, also played a role in shaping investor sentiment.

**Conclusion:**

The decline in Amazon's stock price during November 2025 appears to be correlated with a combination of factors, including broader market concerns, increased competition in key business areas, and investor uncertainty about the company's AI investments. While positive news about AWS growth and strategic partnerships provided some support, the overall trend suggests a cautious outlook among investors.


In [36]:
send_message("What is Apple's ticker overview")

send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response


Upsert chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response


#### Natural language response

Apple Inc. (AAPL) is a technology company based in Cupertino, CA. It was founded on December 12, 1980, and its homepage is https://www.apple.com. Apple is among the largest companies in the world, with a broad portfolio of hardware and software products targeted at consumers and businesses. Apple's iPhone makes up a majority of the firm sales, and Apple's other products like Mac, iPad, and Watch are designed around the iPhone as the focal point of an expansive software ecosystem. Apple has progressively worked to add new applications, like streaming video, subscription bundles, and augmented reality. The firm designs its own software and semiconductors while working with subcontractors like Foxconn and TSMC to build its products and chips. Slightly less than half of Apple's sales come directly through its flagship stores, with a majority of sales coming indirectly through partnerships and distribution. The company has 166,000 employees. Its primary exchange is XNAS. The company's market cap is 4,120,386,034,050.00.

In [37]:
send_message("What is Google's stock ticker symbol?")

send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response


Score wiki search by similarity to topic: 0it [00:00, ?it/s]
Generate wiki embeddings: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response


#### Natural language response

Google's stock ticker symbols on the NASDAQ stock exchange are GOOGL and GOOG. These symbols now refer to Alphabet Inc., which is Google's holding company, since the fourth quarter of 2015. Google is also listed on the Frankfurt Stock Exchange under the ticker symbol GGQ1.

In [38]:
send_message("What is MGM Studio's stock symbol?")

Api.generation_fail.next_model: model is now gemini-2.5-flash-preview-09-2025
send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response


Score wiki search by similarity to topic: 0it [00:00, ?it/s]
Generate wiki embeddings: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response
send_message: get function response


Generate grounding embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response


#### Natural language response

MGM Studios (Metro-Goldwyn-Mayer Studios, Inc.) is a wholly-owned subsidiary of Amazon and is not publicly traded, so it does not have its own stock symbol.

The stock symbol for its parent company, Amazon, is **AMZN**.

In [39]:
send_message("What is MGM Studio's owner company stock symbol?")

send_message: get function response


Score wiki search by similarity to topic: 0it [00:00, ?it/s]
Generate wiki embeddings: 0it [00:00, ?it/s]


send_message: updating state
Api.generation_fail.next_model: model is now gemini-2.0-flash-exp
send_message: got a response
send_message: get function response
send_message: updating state
send_message: got a response


#### Natural language response

The owner of MGM Studios is Amazon, and its stock symbol is AMZN.


In [40]:
send_message("What is Facebook's stock ticker symbol?")

send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response


Score wiki search by similarity to topic: 0it [00:00, ?it/s]
Generate wiki embeddings: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response


#### Natural language response

Facebook's stock ticker symbol is META.


In [41]:
send_message("""Compare Amazon's bullish versus bearish predictions from Oct 01 2025 until today. 
Include a discussion of recommendation trends, and sentiment analysis of news from the same dates. 
Discuss any patterns or correlations you find.""")

send_message: get function response


Upsert chunks embedding: 0it [00:00, ?it/s]


send_message: get function response


Add chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
Api.zero_error: model is now gemini-2.5-flash
send_message: got a response


#### Natural language response

From October 1, 2025, to December 1, 2025, here's a comparison of bullish versus bearish predictions for Amazon, incorporating recommendation trends and sentiment analysis of news:

**Recommendation Trends:**

*   **Overall Bullish Stance:** Analyst recommendations for Amazon have been consistently positive.
*   **Strong Buy and Buy Recommendations:** Strong buy and buy recommendations have remained high, indicating a generally favorable outlook.
*   **Hold Recommendations:** Hold recommendations have been relatively low, suggesting that most analysts have a clear buy or strong buy opinion.
*   **Recommendation Trends:**
    *   December 1, 2025: 52 Buys, 5 Holds, 0 Sells, 21 Strong Buys, 0 Strong Sells
    *   November 1, 2025: 54 Buys, 2 Holds, 0 Sells, 22 Strong Buys, 0 Strong Sells
    *   October 1, 2025: 52 Buys, 3 Holds, 0 Sells, 23 Strong Buys, 0 Strong Sells

**Sentiment Analysis of News:**

*   **Predominantly Positive Sentiment:** The news sentiment for Amazon during this period is largely positive, driven by its strong position in e-commerce, cloud computing (AWS), and AI.
*   **AI as a Growth Driver:** Many articles highlight Amazon's AI initiatives, particularly within AWS, as a significant growth engine.
*   **Strategic Partnerships:** Amazon's partnerships, such as the one with OpenAI, are viewed positively.
*   **Competitive Landscape:** There are mentions of increasing competition in cloud computing and AI, particularly from Microsoft and Google.
*   **Job Cuts:** Some articles mention planned job cuts at Amazon, which could be seen as a cost-cutting measure to improve efficiency or a sign of broader economic concerns.
*   **Market Correction Concerns:** Some articles express concerns about a potential market correction or AI bubble, which could impact Amazon's stock.

**Patterns and Correlations:**

*   **Positive Recommendations and News:** The consistently positive analyst recommendations align with the generally positive news sentiment, suggesting a correlation between expert opinions and media coverage.
*   **AI Focus:** Both recommendation trends and news sentiment emphasize the importance of AI as a key factor influencing Amazon's future performance.
*   **Market Volatility:** Despite the positive outlook, there are recurring concerns about market volatility and potential corrections, indicating that even bullish predictions come with caveats.
*   **Competitive Pressures:** While Amazon is seen as a leader in many areas, the news sentiment also acknowledges increasing competition, which could temper some of the bullish expectations.

In summary, the overall picture for Amazon from October 1, 2025, to December 1, 2025, is positive, with strong analyst recommendations and generally favorable news sentiment. However, it's important to consider the potential risks associated with market volatility, increasing competition, and the possibility of an AI bubble.


In [42]:
send_message("""Compare Google's bullish versus bearish predictions from Oct 01 2025 until today. 
Include a discussion of recommendation trends, and sentiment analysis of news from the same dates. 
Discuss any patterns or correlations you find.""")

send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response


Generate grounding embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response
send_message: get function response


Upsert chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response
send_message: get function response


Add chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
Api.generation_fail.next_model: model is now gemini-2.5-flash-preview-09-2025
send_message: got a response


#### Natural language response


### Google's Bullish vs. Bearish Predictions (October 1, 2025 - December 1, 2025)

The analysis of analyst recommendations and news sentiment for Alphabet Inc. (GOOGL) from October 1, 2025, to December 1, 2025, reveals a consistently **bullish** outlook, driven almost entirely by the company's leadership and strategic investments in Artificial Intelligence (AI) and cloud computing.

#### 1. Recommendation Trends (Bullish)

Analyst recommendations for GOOGL remained overwhelmingly positive throughout the period, with a clear trend toward increasing "Buy" and "Strong Buy" ratings.

| Period | Strong Buy | Buy | Hold | Sell | Strong Sell | Bullish Predictions (Strong Buy + Buy) |
| :--- | :--- | :--- | :--- | :--- | :--- | :--- |
| **December 1, 2025** | 20 | 43 | 10 | 0 | 0 | **63** |
| **November 1, 2025** | 21 | 41 | 12 | 0 | 0 | **62** |
| **October 1, 2025** | 21 | 39 | 13 | 0 | 0 | **60** |

**Key Observations:**
*   **Dominant Bullish Sentiment:** Bullish ratings (Strong Buy + Buy) consistently accounted for over **80%** of all recommendations.
*   **Increasing Confidence:** The number of "Buy" and "Strong Buy" ratings steadily increased from 60 in October to 63 in December, while "Hold" ratings decreased, indicating growing analyst confidence in the stock's performance.
*   **Zero Bearish Ratings:** There were **zero** "Sell" or "Strong Sell" recommendations, suggesting a near-unanimous positive view on the company's fundamentals and future prospects.

#### 2. News Sentiment Analysis (Overwhelmingly Bullish)

The news sentiment for GOOGL during this period was overwhelmingly positive, with a strong focus on AI and strategic business wins.

| Sentiment Category | Key Themes |
| :--- | :--- |
| **Bullish Drivers** | **AI Leadership:** Launch of Gemini 3, development of custom Tensor Processing Units (TPUs) to challenge Nvidia, and successful AI integration across Search and Cloud. **Strategic Investments:** Massive capital expenditure announcements (e.g., $40 billion in Texas, $6.4 billion in Germany) for AI infrastructure. **Investor Confidence:** Significant new investment from Warren Buffett's Berkshire Hathaway (over $4 billion), viewing GOOGL as an undervalued AI play. **Financial Strength:** Record-breaking Q3 earnings (first $100 billion quarter), strong revenue growth (16% YoY), and robust Google Cloud performance (34% revenue growth). |
| **Bearish/Neutral Concerns** | **Competition:** New competitive threats from OpenAI's ChatGPT Atlas browser and custom AI chips from rivals like Amazon and Microsoft. **Regulatory/Legal:** Lawsuit from New York City over social media child addiction (GOOGL is a co-defendant) and ongoing EU regulatory scrutiny. **Valuation/Technical:** Brief mentions of the stock being "technically overbought" (RSI above 79) and potential short-term pullbacks. |

**Overall Sentiment:** The positive news volume and impact far outweighed the negative. The market largely dismissed competitive and regulatory concerns, focusing instead on Alphabet's successful AI monetization and strategic positioning.

#### 3. Patterns and Correlations

A strong **positive correlation** exists between the analyst recommendations and the news sentiment, driven by a few key patterns:

*   **AI Monetization as the Primary Catalyst:** The most significant pattern is the direct link between **successful AI execution** and **bullish sentiment**. News articles repeatedly highlighted Alphabet's ability to monetize AI through its core advertising business (AI-enhanced search) and its rapidly growing, profitable Google Cloud division. This contrasts with competitors like Meta, whose stock tumbled after announcing massive AI spending without clear, immediate monetization.
*   **The "Buffett Effect" and Valuation:** Warren Buffett's decision to invest over $4 billion in Alphabet in Q3 was a major bullish signal, reinforcing the narrative that GOOGL was the **"cheapest Magnificent Seven stock"** and an undervalued AI leader. This institutional endorsement further fueled the positive analyst outlook.
*   **Infrastructure and Vertical Integration:** The news consistently praised Alphabet's strategy of **vertical integration**—developing its own custom AI chips (TPUs) and investing billions in data centers. This was seen as a long-term competitive advantage, reducing reliance on external suppliers like Nvidia and providing a cost-efficient path to AI dominance.
*   **Resilience to Competition:** While new competitors like OpenAI's Atlas browser and rival cloud services were mentioned, the news sentiment generally portrayed Alphabet as resilient, leveraging its massive ecosystem (Chrome, Android, Search) and superior financial resources to counter threats.

In summary, the period from October 1 to December 1, 2025, was characterized by a **highly bullish consensus** for Google, with analyst recommendations and news sentiment converging on the theme of Alphabet as a well-capitalized, strategically positioned **AI powerhouse** successfully translating its technological lead into financial results.

In [43]:
send_message("""How is the outlook for Apple based on trends and news sentiment from July 01 2025 until today? 
Perform the same analysis on all peers by sub-industry. Then compare Apple result to it's peers.""")

send_message: get function response
send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response


Add chunks embedding: 0it [00:00, ?it/s]


send_message: get function response
send_message: get function response
send_message: updating state
Api.zero_error: model is now gemini-2.5-flash
send_message: got a response
send_message: get function response


Add chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response
send_message: get function response


...

send_message: updating state
send_message: got a response
send_message: get function response
send_message: updating state
Api.refill_rpm 10
send_message: got a response
send_message: get function response


Add chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response
send_message: get function response
send_message: updating state
Api.refill_rpm 10
send_message: got a response


#### Natural language response

Here's an outlook for Apple and its peers based on news sentiment and recommendation trends from July 01, 2025, until November 28, 2025:

**Apple (AAPL)**

*   **Recommendation Trends:** Analysts maintain a highly positive outlook for Apple. As of December 01, 2025, recommendations include 15 "Strong Buy," 23 "Buy," 16 "Hold," and only 2 "Sell" ratings. This trend has remained consistently positive throughout the period.
*   **News Sentiment:**
    *   **Positive:** Apple benefits from strong iPhone sales (especially the iPhone 17 series), a rapidly growing Services segment (App Store, Apple TV+, music streaming), and strategic investments in U.S. manufacturing and AI infrastructure. Partnerships with companies like MP Materials for rare earth magnets and potential collaborations with Google for Siri AI are also seen positively. The company received FDA approval for hypertension alerts on Apple Watch, highlighting its leadership in wearable health technology. Strong financial performance, consistent dividend growth, and significant share buybacks contribute to a positive sentiment.
    *   **Negative:** Concerns persist regarding Apple's pace of AI innovation, with delays in Siri AI features and a perceived lag behind competitors. The company faces multiple antitrust lawsuits in various regions (India, China, EU, U.S.) and potential impacts from trade tariffs, particularly on Chinese manufacturing. Warren Buffett's reduction of his stake in Apple has also been noted as a negative signal. Class action lawsuits alleging misleading statements about AI features add to the negative sentiment.
    *   **Neutral:** Mentions in news often relate to its inclusion in ETFs or market indices, or its role in partnerships where the direct impact on Apple isn't explicitly positive or negative.

*   **Overall Outlook for Apple:** Apple's outlook is generally positive, underpinned by its robust ecosystem, strong brand loyalty, and consistent financial performance. The company is actively addressing its AI strategy through internal development and potential acquisitions, and it's diversifying its manufacturing to mitigate tariff risks. However, the pace of AI innovation and ongoing regulatory challenges remain key areas of scrutiny.

**Apple's Peers (Sub-Industry: Computer Hardware)**

Here's a summary of the outlook for Apple's peers in the Computer Hardware sub-industry:

1.  **Dell Technologies (DELL)**
    *   **Recommendation Trends:** Very positive, with 8 "Strong Buy," 16 "Buy," 7 "Hold," and 1 "Sell" as of December 01, 2025.
    *   **News Sentiment:** Highly positive, driven by strong AI server demand, impressive Q3 earnings, and optimistic guidance for fiscal 2026. Strategic partnerships in AI infrastructure and a commitment to annual dividend hikes further bolster its outlook. Some negative sentiment arose from a Morgan Stanley downgrade due to rising memory costs.
    *   **Outlook:** Very positive, benefiting directly from the AI infrastructure boom.

2.  **Western Digital Corporation (WDC)**
    *   **Recommendation Trends:** Consistently positive, with 6 "Strong Buy," 19 "Buy," and 6 "Hold" as of December 01, 2025.
    *   **News Sentiment:** Very strong positive, marked by exceptional stock performance (263% total return in 2025), significant revenue growth, and a 25% dividend increase. High demand for data storage from AI companies is a major driver. Some technical indicators suggest it might be overbought.
    *   **Outlook:** Highly positive, a key beneficiary of AI-driven data storage demand.

3.  **SanDisk Corp (SNDK)**
    *   **Recommendation Trends:** Positive, with 7 "Strong Buy," 11 "Buy," and 6 "Hold" as of December 01, 2025.
    *   **News Sentiment:** Very strong positive, with a remarkable 512% stock price increase in 2025 and inclusion in the S&P 500. Strong performance in NAND flash memory and data center segments, fueled by AI demand, contributes to a bullish outlook.
    *   **Outlook:** Highly positive, benefiting from AI-driven storage demand and a recovering memory market.

4.  **Pure Storage Inc - Class A (PSTG)**
    *   **Recommendation Trends:** Positive, with 7 "Strong Buy," 13 "Buy," and 6 "Hold" as of December 01, 2025.
    *   **News Sentiment:** Positive, outperforming Nvidia in 2025 with strong annual recurring revenue growth and key clients in the AI space. Its subscription-based model is a positive factor.
    *   **Outlook:** Positive, well-positioned for growth in AI data storage with a strong business model.

5.  **Hewlett Packard Enterprise (HPE)**
    *   **Recommendation Trends:** Moderately positive, with 6 "Strong Buy," 7 "Buy," and 13 "Hold" as of December 01, 2025.
    *   **News Sentiment:** Moderately positive, driven by the successful acquisition of Juniper Networks and collaborations in AI and 5G infrastructure. However, restructuring costs have impacted margins.
    *   **Outlook:** Moderately positive, with strategic moves in AI and networking, but some financial pressures.

6.  **NetApp Inc (NTAP)**
    *   **Recommendation Trends:** Moderately positive, with 3 "Strong Buy," 10 "Buy," and 14 "Hold" as of December 01, 2025.
    *   **News Sentiment:** Moderately positive, involved in infrastructure modernization and data protection, with a focus on sustainability. No significant negative news was reported.
    *   **Outlook:** Moderately positive, but analysts are more cautious, reflected in a higher proportion of "Hold" ratings.

7.  **Super Micro Computer Inc (SMCI)**
    *   **Recommendation Trends:** Mixed, with 4 "Strong Buy," 10 "Buy," 9 "Hold," and 3 "Sell" as of December 01, 2025.
    *   **News Sentiment:** Mixed, benefiting from strong AI order backlogs and partnerships with Nvidia, but facing challenges with missed earnings, margin compression, insider selling, and legal allegations.
    *   **Outlook:** Mixed, with high growth potential in AI infrastructure balanced by significant financial and operational risks.

8.  **IonQ Inc (IONQ)**
    *   **Recommendation Trends:** Positive, with 2 "Strong Buy," 10 "Buy," and 4 "Hold" as of December 01, 2025.
    *   **News Sentiment:** Mixed but potentially promising. It shows strong revenue growth and strategic acquisitions in quantum computing, but is unprofitable, highly valued, and faces risks associated with the nascent stage of the technology.
    *   **Outlook:** Highly speculative, with long-term potential in quantum computing but significant near-term risks.

9.  **Quantum Computing Inc (QUBT)**
    *   **Recommendation Trends:** Positive, with 2 "Strong Buy," 5 "Buy," and 2 "Hold" as of December 01, 2025.
    *   **News Sentiment:** Mixed but potentially promising. Strong Q3 earnings and technological innovation are positive, but the company has very low revenue, high operating costs, and an extremely high valuation.
    *   **Outlook:** Highly speculative, with early technological promise but substantial financial and market risks.

**Comparison of Apple to its Peers:**

*   **Market Position and Maturity:** Apple is a highly established, mature market leader with a diversified revenue base and a strong ecosystem. Its peers in computer hardware (Dell, WDC, SNDK, PSTG, HPE, NTAP, SMCI) are also established but operate in more specialized segments, directly benefiting from the AI infrastructure buildout. The quantum computing peers (IonQ, QUBT) are early-stage, highly speculative companies.
*   **AI Strategy:** Apple's AI strategy is perceived as more cautious and, at times, lagging, focusing on device-based AI and privacy. Many of its hardware-focused peers are direct enablers of the AI boom, providing the essential infrastructure. Quantum computing peers are aiming for a future where their technology fundamentally changes AI.
*   **Financial Health & Valuation:** Apple demonstrates strong financial health, consistent profitability, and significant shareholder returns (dividends, buybacks). While some peers like Dell and Western Digital also show strong financial performance and shareholder returns, many, especially in quantum computing, are unprofitable and carry very high valuations relative to their revenue, making them riskier.
*   **Challenges:** Apple faces unique challenges with antitrust regulations and the need to accelerate its AI innovation to maintain its premium valuation. Its peers face challenges related to supply chain, competition, and the inherent risks of emerging technologies (for quantum computing).

**Conclusion:**

Apple's outlook is positive, driven by its strong brand, ecosystem, and growing services. However, it needs to demonstrate more aggressive and successful AI innovation to maintain its competitive edge and justify its valuation, especially as some of its peers are more directly capitalizing on the current AI infrastructure boom. The hardware and storage providers are showing very strong performance due to AI demand, while the quantum computing companies are high-risk, high-reward investments in a nascent but potentially transformative field.

In [44]:
send_message("""What does the recent news say about Apple and the impact of tariffs? From 2025-09-01 up to today. 
Also locate candlestick data for the same dates. 
Discuss in detail any correlations in patterns between the candlestick and news data. Ignore duplicate news entry.""")

send_message: get function response
send_message: generated wrong function arguments
Api.generation_fail.next_model: model is now gemini-2.5-flash-preview-09-2025
send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response


Upsert chunks embedding: 0it [00:00, ?it/s]


send_message: get function response


Add chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response
send_message: get function response
send_message: updating state
Api.zero_error: model is now gemini-2.5-flash
send_message: got a response


#### Natural language response

The stock ticker symbol for Apple is **AAPL**.

Here is a detailed analysis of the recent news regarding Apple and the impact of tariffs, correlated with the stock's candlestick data from **September 1, 2025, to December 1, 2025**.

### Candlestick Data for Apple (AAPL)

| Date | Open (O) | High (H) | Low (L) | Close (C) |
| :--- | :--- | :--- | :--- | :--- |
| **Tue Sep 02, 2025** | $229.25 | $230.85 | $226.97 | $229.72 |
| **Fri Sep 19, 2025** | $241.23 | $246.30 | $240.21 | $245.50 |
| **Fri Oct 10, 2025** | $254.94 | $256.38 | **$244.00** | **$245.27** |
| **Mon Oct 13, 2025** | $249.38 | $249.69 | $245.56 | $247.66 |
| **Mon Oct 27, 2025** | $264.88 | $269.12 | $264.65 | **$268.81** |
| **Fri Oct 31, 2025** | $276.99 | $277.32 | $269.16 | $270.37 |
| **Mon Nov 17, 2025** | $268.82 | $270.49 | $265.73 | **$267.46** |
| **Fri Nov 28, 2025** | $277.26 | $279.00 | $275.99 | $278.85 |

*(Note: Prices are adjusted for splits and dividends. Dates are converted to local time.)*

***

### Recent News on Apple and Tariffs

The news coverage on Apple during this period presented a mixed but ultimately resilient picture regarding tariffs, often intertwined with broader market and AI narratives.

| Date (Local) | News Summary & Sentiment |
| :--- | :--- |
| **Sep 4, 2025** | **Negative/Neutral:** News mentioned Apple is "Facing challenges with AI features and **potential tariffs**," indicating that tariff threats were a known risk factor. |
| **Oct 10, 2025** | **Negative (Market Shock):** Reports of **Trump's renewed tariff threats against China** triggered a significant market selloff, with Apple experiencing a substantial price drop as part of the broader tech sector decline. |
| **Oct 26, 2025** | **Negative (Supply Chain Risk):** An article questioned if Apple would be "Hit Hard by President Trump's Tariffs," highlighting the **risk of potential 100% tariffs** and export restrictions on rare earth elements critical for iPhone components. |
| **Oct 28, 2025** | **Positive (Mitigation Success):** News highlighted Apple's success in **"Powering Through Trump’s Tariff Policies"** by strategically relocating iPhone production and securing exemptions from Chinese and Indian tariffs. |
| **Nov 17, 2025** | **Negative (Confirmed Cost):** A report confirmed the financial burden, stating Apple had already reported **$1.1 billion in tariff-related cost increases**, with expectations for this figure to rise. |
| **Nov 18, 2025** | **Negative (Confirmed Cost):** Another report reiterated the warning about President Trump's tariffs, specifically citing Apple's **$1.1 billion in tariff-related cost increases**. |

***

### Correlation Between Candlestick and News Data

The stock's performance shows a clear correlation with the evolving narrative around tariffs, demonstrating that **investor sentiment was highly sensitive to immediate threats but quickly recovered based on Apple's strategic resilience.**

#### 1. Immediate Negative Shock (October 10)

*   **Candlestick Action:** On **Friday, October 10**, following news of renewed tariff threats, Apple's stock experienced a sharp selloff. It opened at $254.94 and plunged to a low of **$244.00**, closing at $245.27. This single-day drop of **$8.77** from the previous close directly correlates with the market-wide panic over the escalating U.S.-China trade tensions.
*   **Correlation:** **Strong Negative.** The immediate, sharp decline in price and volume spike confirms that the tariff threat was a major catalyst for a market correction.

#### 2. Strategic Resilience and Recovery (Late October)

*   **Candlestick Action:** After the low on October 10, the stock began a steady recovery, culminating in a strong close of **$268.81** on **Monday, October 27**.
*   **News Action:** This recovery aligns perfectly with the narrative shift in the news. On **October 28**, reports confirmed that Apple had successfully mitigated the worst-case scenario by relocating production and securing exemptions. The market was essentially pricing in this successful risk management strategy in the preceding weeks.
*   **Correlation:** **Strong Positive.** The stock's upward trend in the latter half of October was supported by the perception that Apple was effectively neutralizing the tariff threat, demonstrating its ability to adapt its global supply chain.

#### 3. Confirmed Financial Impact (Mid-November)

*   **Candlestick Action:** On **Monday, November 17**, the stock closed at **$267.46**, a slight drop from the previous close, following news that Apple had confirmed **$1.1 billion in tariff-related costs**.
*   **Correlation:** **Weak Negative.** While the news confirmed a significant financial burden, the stock's reaction was muted compared to the initial threat in October. This suggests that the market had already factored in some level of cost or was more focused on other positive drivers, such as the strong iPhone 17 sales and the company's AI strategy, which dominated the news during this period. The overall upward trend from September to December remained intact, indicating that the market viewed the tariff costs as manageable rather than catastrophic.

In summary, the candlestick data for Apple shows that **tariff threats caused immediate, sharp volatility (October 10)**, but the stock's **long-term trajectory was sustained by the company's successful strategic mitigation efforts** and the broader positive sentiment around its product and AI ecosystem.

# StockChat: Agents Edition

## Setup working directory (Kaggle)

In [45]:
# Setup working directory on Kaggle.
if os.getenv("KAGGLE_KERNEL_RUN_TYPE"):
    if not os.path.isdir("sc2/"):
        !git init -b main
        !git remote add origin https://github.com/lol-dungeonmaster/kaggle-agents-2025.git
        !git config core.sparseCheckout true
        !echo "sc2/" >> .git/info/sparse-checkout
        !git pull origin main
        for api_key in ["GOOGLE_API_KEY","POLYGON_API_KEY","FINNHUB_API_KEY"]:
            env_key = UserSecretsClient().get_secret(api_key)
            !echo "$api_key=$env_key" >> sc2/.env # from .venv on local runs
            os.environ[api_key] = UserSecretsClient().get_secret(api_key) # from .venv on local runs

Initialized empty Git repository in /kaggle/working/.git/
remote: Enumerating objects: 563, done.
remote: Counting objects: 100% (260/260), done.
remote: Compressing objects: 100% (146/146), done.
remote: Total 563 (delta 165), reused 204 (delta 114), pack-reused 303 (from 1)
Receiving objects: 100% (563/563), 589.51 KiB | 14.74 MiB/s, done.
Resolving deltas: 100% (357/357), done.
From https://github.com/lol-dungeonmaster/kaggle-agents-2025
 * branch            main       -> FETCH_HEAD
 * [new branch]      main       -> origin/main


## Define the async runner

In [55]:
# Define async runner and helper functions.
from sc2.agent import app
from sc2.src import log
# Logger access not possible on Kaggle to prevent hiding errors.
# - The StderrToLog wrapper will not work as kaggle-docker makes the file-descriptor constant.
# - Redirect basic logger output on Kaggle using print().
if os.getenv("KAGGLE_KERNEL_RUN_TYPE"):
    log_info = print
    log_warn = print
    log_err = print
else:
    log_info = log.info
    log_warn = log.warning
    log_err = log.error

# Display the user query and response after the response is complete.
async def on_event(e: Event, q: str):
    try:
        response = e.content.parts[0].text
        if response and response != "None":
            log_info(f"USER  > {q}")
            log_info(f"MODEL > {response}\n")
    except Exception as err:
        log_err(f"on_event.exception: {str(err)}")

# Run an App with the provided BaseSessionService and user queries list.
async def run_queries(app: App, sessions: SessionService, queries: list[str],
                      session_id: str = "default", user_id: str = "default"):
    runner = Runner(app=app, session_service=sessions)
    try:
        session = await sessions.create_session(
            app_name=runner.app_name, user_id=user_id, session_id=session_id
        )
    except:
        session = await sessions.get_session(
            app_name=runner.app_name, user_id=user_id, session_id=session_id
        )
    finally:
        log_info(f"### Agent session: (uid={user_id}) {session_id}\n")
        for query in queries:
            await try_run(runner, session, user_id, query)

# Launch a runner with the provided session and user_id then respond to query.
# - retries on exceptions TypeError, KeyError, IndexError
async def try_run(runner: Runner, session: Session, user_id: str, query: str):
    try:
        q = types.Content(role="user", parts=[types.Part(text=query)])
        async for response in runner.run_async(
            user_id=user_id, session_id=session.id, new_message=q
        ): await on_event(response, query)
    except Exception as e:
        q_id = " ".join(query.split()[:4])
        if type(e) in [TypeError, KeyError, IndexError]:
            log_warn(f"try_run.run_async (q={q_id}): retrying, generated {type(e).__name__}\n")
            time.sleep(15.0)
            await try_run(runner, session, user_id, query)
        else:
            log_warn(f"try_run.run_async (q={q_id}): {type(e).__name__} - {str(e)}\n")

# Check for compaction events in the provided BaseSessionService.
# - optionally also show the llm compacted output.
async def check_compaction(sessions: SessionService, session_id: str = "default", 
                           user_id: str = "default", show_llm: bool = False):
    n = 0
    for e in (await sessions.get_session(
        app_name=app.name,
        user_id=user_id,
        session_id=session_id,
    )).events:
        if e.actions and (llm_out := e.actions.compaction):
            n += 1
            if show_llm:
                log_info(f"check_compaction.show_llm: {llm_out.compacted_content.parts[0].text}\n")
    log_info(f"check_compaction: found ({n}) compaction event\n")

## Test the async runner

In [47]:
# Create a session service and run some test queries.
s_svc = InMemorySessionService()

await run_queries(
    app=app, sessions=s_svc,
    queries=[
        "What tools do you know how to use?",
        "Ask `sc2_fnplan` what functions fncall_pipeline knows.",
        "What is a short trade?",
        "What is gambler's ruin?",
        "My local advisor is SC at JPMorgan Chase, 212-736-2001",
        "I live in Brooklyn, New York."])

### Agent session: (uid=default) default

USER  > What tools do you know how to use?
MODEL > I know how to use the following tools:

*   **sc2\_memory**: An expert writer of long-term memories.
*   **sc2\_prefs**: An expert profile analyst in the field of finance, money, and stock markets.
*   **fncall\_pipeline**: A function caller with functions defined in sub-agent `sc2_fnplan`.
*   **sc2\_fnplan**: A highly intelligent FunctionTool call planner.
*   **sc2\_terms**: An expert terminologist in the field of finance, money, and stock markets.
*   **sc2\_summary**: An expert proof-reader and writer that knows HTML, JSON and Markdown.

USER  > Ask `sc2_fnplan` what functions fncall_pipeline knows.
MODEL > `fncall_pipeline` knows how to use the following functions:

*   **get\_symbol\_1**: Search for the stock ticker symbol of a given company, security, ISIN, or CUSIP.
*   **get\_symbols\_1**: List all supported symbols and tickers, filtered by exchange code.
*   **get\_name\_1**: Search 

## Test long-term memory

In [48]:
# Use long-term memory from a new session.
s_svc2 = InMemorySessionService()

await run_queries(
    app=app, sessions=s_svc2,
    queries=[
        "Check memory for where I live.",
        "Check memory for my local advisor SCs phone number."])

### Agent session: (uid=default) default

USER  > Check memory for where I live.
MODEL > I remember you live in Brooklyn, New York.

USER  > Check memory for my local advisor SCs phone number.
MODEL > SC's phone number is 212-736-2001. SC is an advisor at JPMorgan Chase.



## Context compaction

In [54]:
# Display context compaction output.
await check_compaction(s_svc, show_llm=True)

check_compaction.show_llm: The conversation covered the AI's capabilities and addressed specific user queries:

*   **AI's Tools**: The AI (`sc2_root`) can use `sc2_memory`, `sc2_prefs`, `fncall_pipeline`, `sc2_fnplan`, `sc2_terms`, and `sc2_summary`.
*   **`fncall_pipeline` Functions**: `fncall_pipeline` has a wide range of functions, primarily focused on financial market data (e.g., `get_symbol_1`, `get_symbol_quote_1`, `get_financials_1`, `get_news_with_sentiment_2`), market status, company information, and general knowledge grounding (`get_wiki_grounding`, `get_search_grounding`).
*   **Definition of "Short Trade"**: The AI provided a detailed explanation of short selling, describing it as an investment strategy anticipating a price decline by borrowing and selling shares, then repurchasing them at a lower price. It also highlighted the high-risk nature of the strategy, including the potential for unlimited losses.

All user questions were answered, and there are no unresolved task

## Run evaluation

In [56]:
!adk eval sc2 sc2/eval/test_cases.json --config_file_path=sc2/eval/config.json --print_detailed_results

/usr/local/lib/python3.11/dist-packages/google/adk/evaluation/metric_evaluator_registry.py:90: UserWarning: [EXPERIMENTAL] MetricEvaluatorRegistry: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  metric_evaluator_registry = MetricEvaluatorRegistry()
/usr/local/lib/python3.11/dist-packages/google/adk/evaluation/local_eval_service.py:80: UserWarning: [EXPERIMENTAL] UserSimulatorProvider: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  user_simulator_provider: UserSimulatorProvider = UserSimulatorProvider(),
Using evaluation criteria: criteria={'rubric_based_final_response_quality_v1': BaseCriterion(threshold=0.8, judge_model_options={'judge_model': 'gemini-2.5-flash', 'num_samples': 1}, rubrics=[{'rubric_id': 'conciseness', 'rubric_content': {'text_property': "The agent's response is direct and to the p